# Matching

## Imports

In [1]:
import numpy as np
import uproot
import vector
import awkward as ak
import matplotlib.pyplot as plt
import mplhep as hep
import boost_histogram as bh
import itertools

directory = "/Users/archiebrooks/Downloads/"
#directory = "/Users/archiebrooks/Documents/Uni/mphys project/MPHYS-4Top/"
#directory = "c:/Users/matis/OneDrive/Documents/Y4/Project/"

hep.style.use('ATLAS')

## Functions and Maps

In [2]:
def histogram(data, bins, data_label, axes, density=False, ratio=False, ratio_axes=None, set_range = None, weight_array=None, x_units='GeV', print_peak=False, ttt_weights=None):
    if len(data[0]) != 1:
        if set_range is not None:
            global_min, global_max = set_range
        else:
            global_min = min([np.min(d) for d in data])
            global_max = max([np.max(d) for d in data])

        bin_edges = np.linspace(global_min, global_max, bins+1)
        counts = []
        errors = []
        bin_width = bin_edges[1]-bin_edges[0]
        for i in range(len(data)):
            if type(data[i])!= 'numpy.ndarray':
                data[i] = np.array(ak.to_numpy(data[i]))
            hist = bh.Histogram(bh.axis.Regular(bins, global_min, global_max))
            if (ttt_weights != None)&(i==1):
                hist.fill(data[i], weight=ttt_weights)
            else:
                hist.fill(data[i], weight=weight_array[i]) if weight_array is not None else hist.fill(data[i])
            norm_factor = np.sum(hist.counts() * np.diff(hist.axes[0].edges))
            if density: hep.histplot(hist.counts()/norm_factor, hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances())/norm_factor,label=data_label[i], histtype='step')
            else: hep.histplot(hist.counts(), hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances()),label=data_label[i], histtype='step')
            counts.append(hist.counts()/norm_factor) if density else counts.append(hist.counts())
            errors.append(np.sqrt(hist.variances())/norm_factor) if density else errors.append(np.sqrt(hist.variances()))       
            if print_peak:
                peak_bin_index = np.argmax(hist.counts())
                peak_bin_center = (hist.axes[0].edges[peak_bin_index] + hist.axes[0].edges[peak_bin_index + 1]) / 2
                print(f"Peak bin center for {data_label[i]}: {peak_bin_center:.2f} {x_units}")
    else:
        hist = bh.Histogram(bh.axis.Regular(bins, global_min, global_max))
        hist.fill(data)
        hep.histplot(hist.view(), hist.axes[0].edges, ax=axes, yerr=np.sqrt(hist.variances()), label=data_label, histtype='step')
        if print_peak:
                peak_bin_index = np.argmax(hist.counts())
                peak_bin_center = (hist.axes[0].edges[peak_bin_index] + hist.axes[0].edges[peak_bin_index + 1]) / 2
                print(f"Peak bin center for {data_label}: {peak_bin_center:.2f} {x_units}")

    bin_width = hist.axes[0].edges[1]-hist.axes[0].edges[0]
    axes.set_ylabel(f'Events /{bin_width:.2g} {x_units}')
    axes.legend()

    if ratio:
        reference_counts = counts[0]
        for i in range(1, len(counts)):
            ratio_values = np.divide(counts[i], reference_counts, out=np.zeros_like(counts[i]), where=reference_counts != 0)
            ratio_errors = np.sqrt(np.divide(errors[i],counts[i], out=np.zeros_like(counts[i]), where=counts[i]!=0)**2 + (np.divide(errors[0],counts[0], out=np.zeros_like(counts[0]), where=counts[0]!=0)**2))
            hep.histplot(ratio_values, bin_edges, ax=ratio_axes, yerr=ratio_errors, label=f'{data_label[i]}/{data_label[0]}')
        ratio_axes.set_ylabel("Ratio")
        ratio_axes.set_xlabel(axes.get_xlabel())
        ratio_axes.axhline(1, color='black', linestyle='--')  # Reference line at ratio=1
        ratio_axes.legend()
        if ratio_axes.get_ylim()[1]>5:
            ratio_axes.set_ylim(0,5)

    hep.atlas.label(ax=axes)

def n_combinations(particles, n, dictionary, type):
    i = n
    while len(particles[ak.num(particles)>=i])!=0:
        particles_i = particles[ak.num(particles)==i]
        if len(particles_i)==0:
            i+=1
            continue
        indices = np.array(list(itertools.combinations(range(np.shape(particles_i)[1]), n)))
        dictionary[type+f'_{i}'] = particles_i[:, indices]
        i += 1
    return None


mass_lookup = {
    1: 2.16E-3,
    2: 4.7E-3,
    3: 1.273,
    4: 93.5E-3,
    5: 172.57,
    6: 4.183,
    11: 0.511E-3,
    12: 0,
    13: 105.66E-3,
    14: 0,
    15: 1.77693,
    16: 0,
    21: 0
}

def duplicate_cleaner(matched_pairs, truth=None):
    truth_ids = matched_pairs['truth']['id']
    deduped_id_pairs = ak.Array([list(dict.fromkeys(row)) for row in ak.to_list(truth_ids)])
    no_repeats = ak.num(deduped_id_pairs) == ak.num(truth_ids)
    one_repeat = ak.num(deduped_id_pairs) == ak.num(truth_ids) - 1

    duplicate_pairs = matched_pairs[one_repeat]
    clean_pairs = matched_pairs[no_repeats]
    if truth is not None:
        duplicate_truth = truth[one_repeat]
        return clean_pairs, duplicate_pairs, duplicate_truth
    
    return clean_pairs, duplicate_pairs, no_repeats

def duplicate_matcher(repeat, dupe_truth):
    fixable_repeats = repeat[ak.num(repeat)>2]

    repeat_truth, valid_particles, duplicate_particles = [], [], []

    print('going into for loop')

    for i in range(1,len(dupe_truth[0])+1):

        repeat_ids = fixable_repeats['truth']['id']
        repeat_i = fixable_repeats[ak.num(repeat_ids[repeat_ids==i], axis=1)==2]
        repeat_ids_i = repeat_ids[ak.num(repeat_ids[repeat_ids==i], axis=1)==2]
        repeat_truth.append(dupe_truth[ak.num(repeat_ids[repeat_ids==i], axis=1)==2])

        valid_particles.append(repeat_i[repeat_ids_i!=i])
        duplicate_particles.append(repeat_i[repeat_ids_i==i])

    print('out of for loop')

    repeat_truth = ak.concatenate(repeat_truth)
    valid_particles = [ak.concatenate(valid_particles)]
    duplicate_particles = ak.concatenate(duplicate_particles)
    id = np.tile(np.arange(1,len(repeat_truth[0])+1), (len(repeat_truth),1))
    repeat_truth_zip = ak.zip({'truth': repeat_truth, 'id': id})

    duplicate_deltars = duplicate_particles['reco'].deltaR(duplicate_particles['truth']['truth'])
    min_dupdeltars = ak.min(duplicate_deltars, axis=1)

    valid_particles.append(duplicate_particles[duplicate_deltars==min_dupdeltars])
    invalid_particles = duplicate_particles['reco'][duplicate_deltars!=min_dupdeltars]

    second_pass = ak.cartesian({'reco': invalid_particles, 'truth': repeat_truth_zip})
    second_deltars = second_pass['reco'].deltaR(second_pass['truth']['truth'])

    second_smallest_dr = ak.sort(second_deltars)[:,1]
    valid_particles.append(second_pass[(second_deltars==second_smallest_dr)&(second_smallest_dr<0.4)])
    print("pre concat")
    valid_particles = ak.concatenate(valid_particles, axis=1)
    print("post concat")
    return valid_particles

def truth_matching(truth, truth_id, reco_jets, reco_leptons):
    id = np.tile(np.arange(1,len(truth[0])+1), (len(truth),1))
    print(len(truth[0]))

    indexed_truth = ak.zip({'truth': truth, 'id': id})
    
    truth_jets = indexed_truth[truth_id < 10]
    truth_leptons = indexed_truth[(truth_id > 10)&(truth_id<20)]

    jets_3d = ak.cartesian({'reco':reco_jets, 'truth': truth_jets}, axis=1, nested=True)

    deltars = jets_3d['reco'].deltaR(jets_3d['truth']['truth'])
    min_deltars = ak.min(deltars,axis=2)

    matched_jets = ak.flatten(jets_3d[(deltars==min_deltars)&(min_deltars<0.4)], axis=2)


    leptons_3d = ak.cartesian({'truth': truth_leptons, 'reco':reco_leptons}, axis=1, nested=True)

    deltars = leptons_3d['reco'].deltaR(leptons_3d['truth']['truth'])
    min_deltars = ak.min(deltars,axis=2)

    matched_leptons = ak.flatten(leptons_3d[(deltars==min_deltars)&(min_deltars<0.1)], axis=2)

    matched = ak.concatenate((matched_jets, matched_leptons), axis=1)

    matched_pairs, repeat_pairs, matched_bools = duplicate_cleaner(matched)

    print(len(repeat_pairs))

    print("starting dm")

    #valid_particles = duplicate_matcher(repeat_pairs, repeat_truth)

    print("starting axis=0 concat")

    #matched_events = ak.concatenate((matched_pairs, valid_particles), axis=0)
    matched_events = matched_pairs
    
    return matched_events, matched_bools

def top_identifier(matched_data, process, lepton_channel, lepton_indices=None):
    """
    Takes the matched, duplicate free data and trys to reconstruct any tops it can
    """

    if process=="4t":
        if lepton_channel == "0L":
            reco_tops_1 = matched_data['reco'][(matched_data['truth']['id']==1)|(matched_data['truth']['id']==5)|(matched_data['truth']['id']==6)]
            reco_tops_2 = matched_data['reco'][(matched_data['truth']['id']==2)|(matched_data['truth']['id']==7)|(matched_data['truth']['id']==8)]
            reco_tops_3 = matched_data['reco'][(matched_data['truth']['id']==3)|(matched_data['truth']['id']==9)|(matched_data['truth']['id']==10)]
            reco_tops_4 = matched_data['reco'][(matched_data['truth']['id']==4)|(matched_data['truth']['id']==11)|(matched_data['truth']['id']==12)]

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==3)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==3)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)
            reco_tops_4 = ak.mask(reco_tops_4, ak.num(reco_tops_4)==3)

        elif lepton_channel == "1L":

            matched_0 = matched_data[(lepton_indices[:,0]==0)]

            reco_tops_1_0 = matched_0['reco'][(matched_0['truth']['id']==1)|(matched_0['truth']['id']==5)]
            reco_tops_2_0 = matched_0['reco'][(matched_0['truth']['id']==2)|(matched_0['truth']['id']==6)|(matched_0['truth']['id']==7)]
            reco_tops_3_0 = matched_0['reco'][(matched_0['truth']['id']==3)|(matched_0['truth']['id']==8)|(matched_0['truth']['id']==9)]
            reco_tops_4_0 = matched_0['reco'][(matched_0['truth']['id']==4)|(matched_0['truth']['id']==10)|(matched_0['truth']['id']==11)]

            matched_2 = matched_data[(lepton_indices[:,0]==2)]
            reco_tops_2_2 = matched_2['reco'][(matched_2['truth']['id']==1)|(matched_2['truth']['id']==5)|(matched_2['truth']['id']==6)]
            reco_tops_1_2 = matched_2['reco'][(matched_2['truth']['id']==2)|(matched_2['truth']['id']==7)]
            reco_tops_3_2 = matched_2['reco'][(matched_2['truth']['id']==3)|(matched_2['truth']['id']==8)|(matched_2['truth']['id']==9)]
            reco_tops_4_2 = matched_2['reco'][(matched_2['truth']['id']==4)|(matched_2['truth']['id']==10)|(matched_2['truth']['id']==11)]

            matched_4 = matched_data[(lepton_indices[:,0]==4)]
            reco_tops_2_4 = matched_4['reco'][(matched_4['truth']['id']==1)|(matched_4['truth']['id']==5)|(matched_4['truth']['id']==6)]
            reco_tops_3_4 = matched_4['reco'][(matched_4['truth']['id']==2)|(matched_4['truth']['id']==7)|(matched_4['truth']['id']==8)]
            reco_tops_1_4 = matched_4['reco'][(matched_4['truth']['id']==3)|(matched_4['truth']['id']==9)]
            reco_tops_4_4 = matched_4['reco'][(matched_4['truth']['id']==4)|(matched_4['truth']['id']==10)|(matched_4['truth']['id']==11)]

            matched_6 = matched_data[(lepton_indices[:,0]==6)]
            reco_tops_2_6 = matched_6['reco'][(matched_6['truth']['id']==1)|(matched_6['truth']['id']==5)|(matched_6['truth']['id']==6)]
            reco_tops_3_6 = matched_6['reco'][(matched_6['truth']['id']==2)|(matched_6['truth']['id']==7)|(matched_6['truth']['id']==8)]
            reco_tops_4_6 = matched_6['reco'][(matched_6['truth']['id']==3)|(matched_6['truth']['id']==9)|(matched_6['truth']['id']==10)]
            reco_tops_1_6 = matched_6['reco'][(matched_6['truth']['id']==4)|(matched_6['truth']['id']==11)]

            reco_tops_1 = ak.concatenate((reco_tops_1_0, reco_tops_1_2, reco_tops_1_4, reco_tops_1_6))
            reco_tops_2 = ak.concatenate((reco_tops_2_0, reco_tops_2_2, reco_tops_2_4, reco_tops_2_6))
            reco_tops_3 = ak.concatenate((reco_tops_3_0, reco_tops_3_2, reco_tops_3_4, reco_tops_3_6))
            reco_tops_4 = ak.concatenate((reco_tops_4_0, reco_tops_4_2, reco_tops_4_4, reco_tops_4_6))

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==2)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==3)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)
            reco_tops_4 = ak.mask(reco_tops_4, ak.num(reco_tops_4)==3)

        elif lepton_channel == "2L":

            matched_01 = matched_data[(lepton_indices[:,0]==0)&(lepton_indices[:,1]==1)]
            reco_tops_1_01 = matched_01['reco'][(matched_01['truth']['id']==1)|(matched_01['truth']['id']==5)]
            reco_tops_2_01 = matched_01['reco'][(matched_01['truth']['id']==2)|(matched_01['truth']['id']==6)]
            reco_tops_3_01 = matched_01['reco'][(matched_01['truth']['id']==3)|(matched_01['truth']['id']==7)|(matched_01['truth']['id']==8)]
            reco_tops_4_01 = matched_01['reco'][(matched_01['truth']['id']==4)|(matched_01['truth']['id']==9)|(matched_01['truth']['id']==10)]

            matched_03 = matched_data[(lepton_indices[:,0]==0)&(lepton_indices[:,1]==3)]
            reco_tops_1_03 = matched_03['reco'][(matched_03['truth']['id']==1)|(matched_03['truth']['id']==5)]
            reco_tops_3_03 = matched_03['reco'][(matched_03['truth']['id']==2)|(matched_03['truth']['id']==6)|(matched_03['truth']['id']==7)]
            reco_tops_2_03 = matched_03['reco'][(matched_03['truth']['id']==3)|(matched_03['truth']['id']==8)]
            reco_tops_4_03 = matched_03['reco'][(matched_03['truth']['id']==4)|(matched_03['truth']['id']==9)|(matched_03['truth']['id']==10)]

            matched_05 = matched_data[(lepton_indices[:,0]==0)&(lepton_indices[:,1]==5)]
            reco_tops_1_05 = matched_05['reco'][(matched_05['truth']['id']==1)|(matched_05['truth']['id']==5)]
            reco_tops_3_05 = matched_05['reco'][(matched_05['truth']['id']==2)|(matched_05['truth']['id']==6)|(matched_05['truth']['id']==7)]
            reco_tops_4_05 = matched_05['reco'][(matched_05['truth']['id']==3)|(matched_05['truth']['id']==8)|(matched_05['truth']['id']==9)]
            reco_tops_2_05 = matched_05['reco'][(matched_05['truth']['id']==4)|(matched_05['truth']['id']==10)]

            matched_23 = matched_data[(lepton_indices[:,0]==2)&(lepton_indices[:,1]==3)]
            reco_tops_3_23 = matched_23['reco'][(matched_23['truth']['id']==1)|(matched_23['truth']['id']==5)|(matched_23['truth']['id']==6)]
            reco_tops_1_23 = matched_23['reco'][(matched_23['truth']['id']==2)|(matched_23['truth']['id']==7)]
            reco_tops_2_23 = matched_23['reco'][(matched_23['truth']['id']==3)|(matched_23['truth']['id']==8)]
            reco_tops_4_23 = matched_23['reco'][(matched_23['truth']['id']==4)|(matched_23['truth']['id']==9)|(matched_23['truth']['id']==10)]

            matched_25 = matched_data[(lepton_indices[:,0]==2)&(lepton_indices[:,1]==5)]
            reco_tops_3_25 = matched_25['reco'][(matched_25['truth']['id']==1)|(matched_25['truth']['id']==5)|(matched_25['truth']['id']==6)]
            reco_tops_1_25 = matched_25['reco'][(matched_25['truth']['id']==2)|(matched_25['truth']['id']==7)]
            reco_tops_4_25 = matched_25['reco'][(matched_25['truth']['id']==3)|(matched_25['truth']['id']==8)|(matched_25['truth']['id']==9)]
            reco_tops_2_25 = matched_25['reco'][(matched_25['truth']['id']==4)|(matched_25['truth']['id']==10)]

            matched_45 = matched_data[lepton_indices[:,0]==4]
            reco_tops_3_45 = matched_45['reco'][(matched_45['truth']['id']==1)|(matched_45['truth']['id']==5)|(matched_45['truth']['id']==6)]
            reco_tops_4_45 = matched_45['reco'][(matched_45['truth']['id']==2)|(matched_45['truth']['id']==7)|(matched_45['truth']['id']==8)]
            reco_tops_1_45 = matched_45['reco'][(matched_45['truth']['id']==3)|(matched_45['truth']['id']==9)]
            reco_tops_2_45 = matched_45['reco'][(matched_45['truth']['id']==4)|(matched_45['truth']['id']==10)]

            reco_tops_1 = ak.concatenate((reco_tops_1_01, reco_tops_1_03, reco_tops_1_05, reco_tops_1_23, reco_tops_1_25, reco_tops_1_45))
            reco_tops_2 = ak.concatenate((reco_tops_2_01, reco_tops_2_03, reco_tops_2_05, reco_tops_2_23, reco_tops_2_25, reco_tops_2_45))
            reco_tops_3 = ak.concatenate((reco_tops_3_01, reco_tops_3_03, reco_tops_3_05, reco_tops_3_23, reco_tops_3_25, reco_tops_3_45))
            reco_tops_4 = ak.concatenate((reco_tops_4_01, reco_tops_4_03, reco_tops_4_05, reco_tops_4_23, reco_tops_4_25, reco_tops_4_45))

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==2)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==2)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)
            reco_tops_4 = ak.mask(reco_tops_4, ak.num(reco_tops_4)==3)

        reco_top_1 = ak.unflatten(ak.sum(reco_tops_1, axis=1), 1)
        reco_top_2 = ak.unflatten(ak.sum(reco_tops_2, axis=1), 1)
        reco_top_3 = ak.unflatten(ak.sum(reco_tops_3, axis=1), 1)
        reco_top_4 = ak.unflatten(ak.sum(reco_tops_4, axis=1), 1)

        all_reco_tops = ak.drop_none(ak.concatenate((reco_top_1,reco_top_2,reco_top_3,reco_top_4), axis=1))

        sm_mask, dm_mask, tm_mask, fm_mask = (ak.num(all_reco_tops)==1), (ak.num(all_reco_tops)==2), (ak.num(all_reco_tops)==3), (ak.num(all_reco_tops)==4)

        return sm_mask,  dm_mask, tm_mask, fm_mask
        
    elif (process=="3tj")|(process=="3tW"):
        if lepton_channel == "0L":
            reco_tops_1 = matched_data['reco'][(matched_data['truth']['id']==1)|(matched_data['truth']['id']==4)|(matched_data['truth']['id']==5)]
            reco_tops_2 = matched_data['reco'][(matched_data['truth']['id']==2)|(matched_data['truth']['id']==6)|(matched_data['truth']['id']==7)]
            reco_tops_3 = matched_data['reco'][(matched_data['truth']['id']==3)|(matched_data['truth']['id']==8)|(matched_data['truth']['id']==9)]

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==3)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==3)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)

        elif lepton_channel == "1L":

            matched_0 = matched_data[(lepton_indices[:,0]==0)]

            reco_tops_1_0 = matched_0['reco'][(matched_0['truth']['id']==1)|(matched_0['truth']['id']==4)]
            reco_tops_2_0 = matched_0['reco'][(matched_0['truth']['id']==2)|(matched_0['truth']['id']==5)|(matched_0['truth']['id']==6)]
            reco_tops_3_0 = matched_0['reco'][(matched_0['truth']['id']==3)|(matched_0['truth']['id']==7)|(matched_0['truth']['id']==8)]

            matched_2 = matched_data[(lepton_indices[:,0]==2)]
            reco_tops_2_2 = matched_2['reco'][(matched_2['truth']['id']==1)|(matched_2['truth']['id']==4)|(matched_2['truth']['id']==5)]
            reco_tops_1_2 = matched_2['reco'][(matched_2['truth']['id']==2)|(matched_2['truth']['id']==6)]
            reco_tops_3_2 = matched_2['reco'][(matched_2['truth']['id']==3)|(matched_2['truth']['id']==7)|(matched_2['truth']['id']==8)]

            matched_4 = matched_data[(lepton_indices[:,0]==4)]
            reco_tops_2_4 = matched_4['reco'][(matched_4['truth']['id']==1)|(matched_4['truth']['id']==4)|(matched_4['truth']['id']==5)]
            reco_tops_3_4 = matched_4['reco'][(matched_4['truth']['id']==2)|(matched_4['truth']['id']==6)|(matched_4['truth']['id']==7)]
            reco_tops_1_4 = matched_4['reco'][(matched_4['truth']['id']==3)|(matched_4['truth']['id']==8)]

            reco_tops_1 = ak.concatenate((reco_tops_1_0, reco_tops_1_2, reco_tops_1_4))
            reco_tops_2 = ak.concatenate((reco_tops_2_0, reco_tops_2_2, reco_tops_2_4))
            reco_tops_3 = ak.concatenate((reco_tops_3_0, reco_tops_3_2, reco_tops_3_4))

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==2)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==3)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)

        elif lepton_channel == "2L":

            matched_01 = matched_data[(lepton_indices[:,0]==0)&(lepton_indices[:,1]==1)]
            reco_tops_1_01 = matched_01['reco'][(matched_01['truth']['id']==1)|(matched_01['truth']['id']==4)]
            reco_tops_2_01 = matched_01['reco'][(matched_01['truth']['id']==2)|(matched_01['truth']['id']==5)]
            reco_tops_3_01 = matched_01['reco'][(matched_01['truth']['id']==3)|(matched_01['truth']['id']==6)|(matched_01['truth']['id']==7)]

            matched_03 = matched_data[(lepton_indices[:,0]==0)&(lepton_indices[:,1]==3)]
            reco_tops_1_03 = matched_03['reco'][(matched_03['truth']['id']==1)|(matched_03['truth']['id']==4)]
            reco_tops_3_03 = matched_03['reco'][(matched_03['truth']['id']==2)|(matched_03['truth']['id']==5)|(matched_03['truth']['id']==6)]
            reco_tops_2_03 = matched_03['reco'][(matched_03['truth']['id']==3)|(matched_03['truth']['id']==7)]

            matched_23 = matched_data[(lepton_indices[:,0]==2)&(lepton_indices[:,1]==3)]
            reco_tops_3_23 = matched_23['reco'][(matched_23['truth']['id']==1)|(matched_23['truth']['id']==4)|(matched_23['truth']['id']==5)]
            reco_tops_1_23 = matched_23['reco'][(matched_23['truth']['id']==2)|(matched_23['truth']['id']==6)]
            reco_tops_2_23 = matched_23['reco'][(matched_23['truth']['id']==3)|(matched_23['truth']['id']==7)]

            reco_tops_1 = ak.concatenate((reco_tops_1_01, reco_tops_1_03, reco_tops_1_23))
            reco_tops_2 = ak.concatenate((reco_tops_2_01, reco_tops_2_03, reco_tops_2_23))
            reco_tops_3 = ak.concatenate((reco_tops_3_01, reco_tops_3_03, reco_tops_3_23))

            reco_tops_1 = ak.mask(reco_tops_1, ak.num(reco_tops_1)==2)
            reco_tops_2 = ak.mask(reco_tops_2, ak.num(reco_tops_2)==2)
            reco_tops_3 = ak.mask(reco_tops_3, ak.num(reco_tops_3)==3)

        reco_top_1 = ak.unflatten(ak.sum(reco_tops_1, axis=1), 1)
        reco_top_2 = ak.unflatten(ak.sum(reco_tops_2, axis=1), 1)
        reco_top_3 = ak.unflatten(ak.sum(reco_tops_3, axis=1), 1)

        all_reco_tops = ak.drop_none(ak.concatenate((reco_top_1,reco_top_2,reco_top_3), axis=1))

        sm_mask, dm_mask, fm_mask = (ak.num(all_reco_tops)==1), (ak.num(all_reco_tops)==2), (ak.num(all_reco_tops)==3)

        return sm_mask,  dm_mask, fm_mask

    else:
        print('Channel not recognised')
        return None, None

## Data

### 4 Top Matching

In [3]:
tree4 = uproot.open(directory + "4tops_parsed_combined_2811.root")
tree4_truth = tree4['Truth;1']
tree4_reco = tree4['Reco;1']

events_4t_truth = tree4_truth.arrays(['b_id', 'b_pt', 'b_eta', 'b_phi', 'b_e', 'b_mass', 'W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])

In [4]:
len(events_4t_truth['b_id'])

10000000

In [10]:
tree4 = uproot.open(directory + "4tops_parsed_combined_2811.root")
tree4_truth = tree4['Truth;1']
tree4_reco = tree4['Reco;1']

events_4t_truth = tree4_truth.arrays(['b_id', 'b_pt', 'b_eta', 'b_phi', 'b_e', 'b_mass', 'W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_4t_t_neut = tree4_truth.arrays(['W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_4t_partonic = tree4_truth.arrays(['top_pt', 'top_eta', 'top_phi', 'top_e', 'top_mass', 'top_id', 'W_pt', 'W_eta', 'W_phi', 'W_e', 'W_mass', 'W_id'])
events_4t_reco = tree4_reco.arrays(['jet_pt', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_btag', 'el_pt', 'el_eta', 'el_phi', 'el_charge', 'mu_pt', 'mu_eta', 'mu_phi', 'mu_charge'])
events_4t_r_neut = tree4_reco.arrays(['met_met', 'met_eta', 'met_phi'])


events_4t_reco['el_mass'] = 0.511e-3 * np.ones_like(events_4t_reco['el_pt'])
events_4t_reco['mu_mass'] = 0.1057 * np.ones_like(events_4t_reco['mu_pt'])
events_4t_reco['jet_e'] = np.sqrt(events_4t_reco['jet_mass']**2+(events_4t_reco['jet_pt']**2)*(np.cosh(events_4t_reco['jet_eta']))**2)
events_4t_reco['el_e'] = np.sqrt(events_4t_reco['el_mass']**2+(events_4t_reco['el_pt']**2)*(np.cosh(events_4t_reco['el_eta']))**2)
events_4t_reco['mu_e'] = np.sqrt(events_4t_reco['mu_mass']**2+(events_4t_reco['mu_pt']**2)*(np.cosh(events_4t_reco['mu_eta']))**2)
events_4t_r_neut['met_e'] = (events_4t_r_neut['met_met'])*(np.cosh(events_4t_r_neut['met_eta']))

good_events_4t = ak.num(events_4t_truth['b_id']) == 4

invalid_events_4t_truth = events_4t_truth[~good_events_4t]
invalid_events_4t_truth['b_id'], invalid_events_4t_truth['b_pt'], invalid_events_4t_truth['b_eta'] = invalid_events_4t_truth['b_id'][:,1:], invalid_events_4t_truth['b_pt'][:,1:], invalid_events_4t_truth['b_eta'][:,1:]
invalid_events_4t_truth['b_phi'], invalid_events_4t_truth['b_e'], invalid_events_4t_truth['b_mass'] = invalid_events_4t_truth['b_phi'][:,1:], invalid_events_4t_truth['b_e'][:,1:], invalid_events_4t_truth['b_mass'][:,1:]

events_4t_truth = ak.concatenate((events_4t_truth[good_events_4t], invalid_events_4t_truth))
events_4t_reco = ak.concatenate((events_4t_reco[good_events_4t], events_4t_reco[~good_events_4t]))
events_4t_partonic = ak.concatenate((events_4t_partonic[good_events_4t], events_4t_partonic[~good_events_4t]))
events_4t_t_neut = ak.concatenate((events_4t_t_neut[good_events_4t], events_4t_t_neut[~good_events_4t]))
events_4t_r_neut = ak.concatenate((events_4t_r_neut[good_events_4t], events_4t_r_neut[~good_events_4t]))

x = events_4t_truth['W_decay_id'][(events_4t_truth['W_decay_id']==15)|(events_4t_truth['W_decay_id']==-15)]
events_4t_reco = events_4t_reco[ak.num(x)==0]
events_4t_partonic = events_4t_partonic[ak.num(x)==0]
events_4t_truth = events_4t_truth[ak.num(x)==0]
events_4t_t_neut = events_4t_t_neut[ak.num(x)==0]
events_4t_r_neut = events_4t_r_neut[ak.num(x)==0]

neutrino_remover = (abs(events_4t_truth['W_decay_id']) != 12)&(abs(events_4t_truth['W_decay_id']) != 14)&(abs(events_4t_truth['W_decay_id']) != 16)&(abs(events_4t_truth['W_decay_id']) != 18)
events_4t_t_neut['W_decay_id'] = events_4t_t_neut['W_decay_id'][~neutrino_remover]
events_4t_t_neut['W_decay_pt'] = events_4t_t_neut['W_decay_pt'][~neutrino_remover]
events_4t_t_neut['W_decay_eta'] = events_4t_t_neut['W_decay_eta'][~neutrino_remover]
events_4t_t_neut['W_decay_phi'] = events_4t_t_neut['W_decay_phi'][~neutrino_remover]
events_4t_t_neut['W_decay_e'] = events_4t_t_neut['W_decay_e'][~neutrino_remover]
events_4t_t_neut['W_decay_mass'] = events_4t_t_neut['W_decay_mass'][~neutrino_remover]

events_4t_truth['W_decay_id'] = events_4t_truth['W_decay_id'][neutrino_remover]
events_4t_truth['W_decay_pt'] = events_4t_truth['W_decay_pt'][neutrino_remover]
events_4t_truth['W_decay_eta'] = events_4t_truth['W_decay_eta'][neutrino_remover]
events_4t_truth['W_decay_phi'] = events_4t_truth['W_decay_phi'][neutrino_remover]
events_4t_truth['W_decay_e'] = events_4t_truth['W_decay_e'][neutrino_remover]
events_4t_truth['W_decay_mass'] = events_4t_truth['W_decay_mass'][neutrino_remover]

jet_pt_4t = events_4t_reco['jet_pt'][(events_4t_reco['jet_eta']<2.5) & (events_4t_reco['jet_eta']>-2.5)]
jet_eta_4t = events_4t_reco['jet_eta'][(events_4t_reco['jet_eta']<2.5) & (events_4t_reco['jet_eta']>-2.5)]
jet_phi_4t = events_4t_reco['jet_phi'][(events_4t_reco['jet_eta']<2.5) & (events_4t_reco['jet_eta']>-2.5)]
jet_mass_4t = events_4t_reco['jet_mass'][(events_4t_reco['jet_eta']<2.5) & (events_4t_reco['jet_eta']>-2.5)]
jet_e_4t = events_4t_reco['jet_e'][(events_4t_reco['jet_eta']<2.5) & (events_4t_reco['jet_eta']>-2.5)]

electron_pt_4t = events_4t_reco['el_pt'][(events_4t_reco['el_pt'] > 15) & (events_4t_reco['el_eta'] < 2.47) & (events_4t_reco['el_eta'] > -2.47)]
electron_eta_4t = events_4t_reco['el_eta'][(events_4t_reco['el_pt'] > 15) & (events_4t_reco['el_eta'] < 2.47) & (events_4t_reco['el_eta'] > -2.47)]
electron_phi_4t = events_4t_reco['el_phi'][(events_4t_reco['el_pt'] > 15) & (events_4t_reco['el_eta'] < 2.47) & (events_4t_reco['el_eta'] > -2.47)]
electron_mass_4t = events_4t_reco['el_mass'][(events_4t_reco['el_pt'] > 15) & (events_4t_reco['el_eta'] < 2.47) & (events_4t_reco['el_eta'] > -2.47)]
electron_e_4t = events_4t_reco['el_e'][(events_4t_reco['el_pt'] > 15) & (events_4t_reco['el_eta'] < 2.47) & (events_4t_reco['el_eta'] > -2.47)]

muon_pt_4t = events_4t_reco['mu_pt'][(events_4t_reco['mu_pt']>15) & (events_4t_reco['mu_eta']<2.5) & (events_4t_reco['mu_eta']>-2.5)]
muon_eta_4t = events_4t_reco['mu_eta'][(events_4t_reco['mu_pt']>15) & (events_4t_reco['mu_eta']<2.5) & (events_4t_reco['mu_eta']>-2.5)]
muon_phi_4t = events_4t_reco['mu_phi'][(events_4t_reco['mu_pt']>15) & (events_4t_reco['mu_eta']<2.5) & (events_4t_reco['mu_eta']>-2.5)]
muon_mass_4t = events_4t_reco['mu_mass'][(events_4t_reco['mu_pt']>15) & (events_4t_reco['mu_eta']<2.5) & (events_4t_reco['mu_eta']>-2.5)]
muon_e_4t = events_4t_reco['mu_e'][(events_4t_reco['mu_pt']>15) & (events_4t_reco['mu_eta']<2.5) & (events_4t_reco['mu_eta']>-2.5)]

lepton_pt_4t = ak.concatenate((muon_pt_4t,electron_pt_4t), axis=1)
lepton_eta_4t = ak.concatenate((muon_eta_4t,electron_eta_4t), axis=1)
lepton_phi_4t = ak.concatenate((muon_phi_4t,electron_phi_4t), axis=1)
lepton_mass_4t = ak.concatenate((muon_mass_4t,electron_mass_4t), axis=1)
lepton_e_4t = ak.concatenate((muon_e_4t,electron_e_4t), axis=1)

reco_pt_4t = ak.concatenate((jet_pt_4t,lepton_pt_4t), axis=1)
reco_eta_4t = ak.concatenate((jet_eta_4t,lepton_eta_4t), axis=1)
reco_phi_4t = ak.concatenate((jet_phi_4t,lepton_phi_4t), axis=1)
reco_mass_4t = ak.concatenate((jet_mass_4t,lepton_mass_4t), axis=1)
reco_e_4t = ak.concatenate((jet_e_4t,lepton_e_4t), axis=1)

b_pt_4t = events_4t_truth['b_pt']
b_eta_4t = events_4t_truth['b_eta']
b_phi_4t = events_4t_truth['b_phi']
b_mass_4t = events_4t_truth['b_mass']
b_e_4t = events_4t_truth['b_e']
b_id_4t = events_4t_truth['b_id']

w_decay_pt_4t = events_4t_truth['W_decay_pt']
w_decay_eta_4t = events_4t_truth['W_decay_eta']
w_decay_phi_4t = events_4t_truth['W_decay_phi']
w_decay_mass_4t = events_4t_truth['W_decay_mass']
w_decay_e_4t = events_4t_truth['W_decay_e']
w_decay_id_4t = events_4t_truth['W_decay_id']

truth_pt_4t = ak.concatenate((b_pt_4t,w_decay_pt_4t), axis=1)
truth_eta_4t = ak.concatenate((b_eta_4t,w_decay_eta_4t), axis=1)
truth_phi_4t = ak.concatenate((b_phi_4t,w_decay_phi_4t), axis=1)
truth_mass_4t = ak.concatenate((b_mass_4t,w_decay_mass_4t), axis=1)
truth_e_4t = ak.concatenate((b_e_4t,w_decay_e_4t), axis=1)
truth_id_4t = ak.concatenate((b_id_4t,w_decay_id_4t), axis=1)

reco_4t = vector.zip({'pt':reco_pt_4t,'eta':reco_eta_4t,'phi':reco_phi_4t,'energy':reco_e_4t})
truth_4t = vector.zip({'pt':truth_pt_4t,'eta':truth_eta_4t,'phi':truth_phi_4t,'energy':truth_e_4t})

reco_4t_leptons = vector.zip({'pt':lepton_pt_4t,'eta':lepton_eta_4t,'phi':lepton_phi_4t,'energy':lepton_e_4t})
reco_4t_jets = vector.zip({'pt':jet_pt_4t,'eta':jet_eta_4t,'phi':jet_phi_4t,'energy':jet_e_4t})

truth_4t = truth_4t[(ak.num(reco_4t)!=0)]
reco_4t = reco_4t[(ak.num(reco_4t)!=0)]
reco_4t_leptons = reco_4t_leptons[(ak.num(reco_4t)!=0)]
reco_4t_jets = reco_4t_jets[(ak.num(reco_4t)!=0)]

In [4]:
len(truth_4t)

6333547

In [5]:
len(events_4t_r_neut)

6333547

### Channel Splitting for 4t

In [11]:
reco_lepton_count_4t = ak.num(reco_4t_leptons)
truth_lepton_count_4t = ak.num(events_4t_truth['W_decay_id'][(abs(events_4t_truth['W_decay_id'])>10)&(abs(events_4t_truth['W_decay_id'])<19)])

mask_0L_4t = (reco_lepton_count_4t == 0) & (truth_lepton_count_4t == 0)
mask_1L_4t = (reco_lepton_count_4t == 1) & (truth_lepton_count_4t == 1)
mask_2L_4t = (reco_lepton_count_4t == 2) & (truth_lepton_count_4t == 2)
mask_3L_4t = (reco_lepton_count_4t == 3) & (truth_lepton_count_4t == 3)
mask_4L_4t = (reco_lepton_count_4t == 4) & (truth_lepton_count_4t == 4)

events_4t_truth_0L, events_4t_reco_0L = events_4t_truth[mask_0L_4t], events_4t_reco[mask_0L_4t]
events_4t_truth_1L, events_4t_reco_1L = events_4t_truth[mask_1L_4t], events_4t_reco[mask_1L_4t]
events_4t_t_neut_1L, events_4t_r_neut_1L = events_4t_t_neut[mask_1L_4t], events_4t_r_neut[mask_1L_4t]
events_4t_truth_2L, events_4t_reco_2L = events_4t_truth[mask_2L_4t], events_4t_reco[mask_2L_4t]
events_4t_truth_3L, events_4t_reco_3L = events_4t_truth[mask_3L_4t], events_4t_reco[mask_3L_4t]
events_4t_truth_4L, events_4t_reco_4L = events_4t_truth[mask_4L_4t], events_4t_reco[mask_4L_4t]

truth_4t_0L, truth_id_4t_0L = truth_4t[mask_0L_4t], truth_id_4t[mask_0L_4t]
truth_4t_1L, truth_id_4t_1L = truth_4t[mask_1L_4t], truth_id_4t[mask_1L_4t]
truth_4t_2L, truth_id_4t_2L = truth_4t[mask_2L_4t], truth_id_4t[mask_2L_4t]
truth_4t_3L, truth_id_4t_3L = truth_4t[mask_3L_4t], truth_id_4t[mask_3L_4t]
truth_4t_4L, truth_id_4t_4L = truth_4t[mask_4L_4t], truth_id_4t[mask_4L_4t]

reco_4t_0L, reco_4t_leptons_0L, reco_4t_jets_0L = reco_4t[mask_0L_4t], reco_4t_leptons[mask_0L_4t], reco_4t_jets[mask_0L_4t]
reco_4t_1L, reco_4t_leptons_1L, reco_4t_jets_1L = reco_4t[mask_1L_4t], reco_4t_leptons[mask_1L_4t], reco_4t_jets[mask_1L_4t]
reco_4t_2L, reco_4t_leptons_2L, reco_4t_jets_2L = reco_4t[mask_2L_4t], reco_4t_leptons[mask_2L_4t], reco_4t_jets[mask_2L_4t]
reco_4t_3L, reco_4t_leptons_3L, reco_4t_jets_3L = reco_4t[mask_3L_4t], reco_4t_leptons[mask_3L_4t], reco_4t_jets[mask_3L_4t]
reco_4t_4L, reco_4t_leptons_4L, reco_4t_jets_4L = reco_4t[mask_4L_4t], reco_4t_leptons[mask_4L_4t], reco_4t_jets[mask_4L_4t]

In [12]:
len(events_4t_r_neut)

6333547

### Matching + Writing

#### 0 Lepton Channel

In [6]:
truth_4t_0L

<MomentumArray4D [[{rho: 56.6, phi: ..., ...}, ...], ...] type='2084613 * v...'>

In [7]:
truth_4t_0L

<MomentumArray4D [[{rho: 56.6, phi: ..., ...}, ...], ...] type='2084613 * v...'>

In [13]:
matched_events_4t_0L, no_dupes_bool_0L = truth_matching(truth_4t_0L, truth_id_4t_0L, reco_4t_jets_0L, reco_4t_leptons_0L)
matched_4t_0L = ak.zip({'reco': matched_events_4t_0L['reco'], 'truth': matched_events_4t_0L['truth']})
sm_mask_4t_0L, dm_mask_4t_0L, tm_mask_4t_0L, fm_mask_4t_0L= top_identifier(matched_4t_0L, '4t', '0L')

12
130150
starting dm
starting axis=0 concat


In [9]:
fm_events_4t_reco = events_4t_reco_0L[no_dupes_bool_0L][fm_mask_4t_0L]
fm_events_4t_truth = events_4t_truth_0L[no_dupes_bool_0L][fm_mask_4t_0L]

fm_truth_4t_0L = {}
fm_reco_4t_0L = {}

for field in fm_events_4t_truth.fields:
    fm_truth_4t_0L[f"{field}"] = fm_events_4t_truth[field]
for field in fm_events_4t_reco.fields:
    fm_reco_4t_0L[f"{field}"] = fm_events_4t_reco[field]

with  uproot.recreate(f"fm_4t_0L_02Dec.root") as file:
       file["Truth"] = fm_truth_4t_0L
       file["Reco"] = fm_reco_4t_0L

tm_events_4t_reco = events_4t_reco_0L[no_dupes_bool_0L][tm_mask_4t_0L]
tm_events_4t_truth = events_4t_truth_0L[no_dupes_bool_0L][tm_mask_4t_0L]

tm_truth_4t_0L = {}
tm_reco_4t_0L = {}

for field in tm_events_4t_truth.fields:
    tm_truth_4t_0L[f"{field}"] = tm_events_4t_truth[field]
for field in tm_events_4t_reco.fields:
    tm_reco_4t_0L[f"{field}"] = tm_events_4t_reco[field]

with  uproot.recreate(f"tm_4t_0L_02Dec.root") as file:
       file["Truth"] = tm_truth_4t_0L
       file["Reco"] = tm_reco_4t_0L

dm_events_4t_reco = events_4t_reco_0L[no_dupes_bool_0L][dm_mask_4t_0L]
dm_events_4t_truth = events_4t_truth_0L[no_dupes_bool_0L][dm_mask_4t_0L]

dm_truth_4t_0L = {}
dm_reco_4t_0L = {}

for field in dm_events_4t_truth.fields:
    dm_truth_4t_0L[f"{field}"] = dm_events_4t_truth[field]
for field in dm_events_4t_reco.fields:
    dm_reco_4t_0L[f"{field}"] = dm_events_4t_reco[field]

with  uproot.recreate(f"dm_4t_0L_02Dec.root") as file:
       file["Truth"] = dm_truth_4t_0L
       file["Reco"] = dm_reco_4t_0L

sm_events_4t_reco = events_4t_reco_0L[no_dupes_bool_0L][sm_mask_4t_0L]
sm_events_4t_truth = events_4t_truth_0L[no_dupes_bool_0L][sm_mask_4t_0L]

sm_truth_4t_0L = {}
sm_reco_4t_0L = {}

for field in sm_events_4t_truth.fields:
    sm_truth_4t_0L[f"{field}"] = sm_events_4t_truth[field]
for field in sm_events_4t_reco.fields:
    sm_reco_4t_0L[f"{field}"] = sm_events_4t_reco[field]

with  uproot.recreate(f"sm_4t_0L_02Dec.root") as file:
       file["Truth"] = sm_truth_4t_0L
       file["Reco"] = sm_reco_4t_0L

#### 1 Lepton Channel

In [10]:
matched_events_4t_1L, no_dupes_bool_1L = truth_matching(truth_4t_1L, truth_id_4t_1L, reco_4t_jets_1L, reco_4t_leptons_1L)

11
starting dm
starting axis=0 concat


In [11]:
matched_4t_1L = ak.zip({'reco': matched_events_4t_1L['reco'], 'truth': matched_events_4t_1L['truth']})
events_4t_truth_1L = events_4t_truth_1L[no_dupes_bool_1L]
events_4t_reco_1L = events_4t_reco_1L[no_dupes_bool_1L]
events_4t_t_neut_1L = events_4t_t_neut_1L[no_dupes_bool_1L]
events_4t_r_neut_1L = events_4t_r_neut_1L[no_dupes_bool_1L]
lepton_indices_1L = ak.sort(ak.argsort(abs(events_4t_truth_1L['W_decay_id']), axis = 1, ascending=False)[:,:1], axis=1)

In [12]:
sm_mask_4t_1L, dm_mask_4t_1L, tm_mask_4t_1L, fm_mask_4t_1L= top_identifier(matched_4t_1L, '4t', '1L', lepton_indices=lepton_indices_1L)

In [13]:
events_4t_truth_0_1L = events_4t_truth_1L[(lepton_indices_1L[:,0]==0)]
events_4t_truth_2_1L = events_4t_truth_1L[(lepton_indices_1L[:,0]==2)]
events_4t_truth_4_1L = events_4t_truth_1L[(lepton_indices_1L[:,0]==4)]
events_4t_truth_6_1L = events_4t_truth_1L[(lepton_indices_1L[:,0]==6)]

events_4t_reco_0_1L = events_4t_reco_1L[(lepton_indices_1L[:,0]==0)]
events_4t_reco_2_1L = events_4t_reco_1L[(lepton_indices_1L[:,0]==2)]
events_4t_reco_4_1L = events_4t_reco_1L[(lepton_indices_1L[:,0]==4)]
events_4t_reco_6_1L = events_4t_reco_1L[(lepton_indices_1L[:,0]==6)]

events_4t_t_neut_0_1L = events_4t_t_neut_1L[(lepton_indices_1L[:,0]==0)]
events_4t_t_neut_2_1L = events_4t_t_neut_1L[(lepton_indices_1L[:,0]==2)]
events_4t_t_neut_4_1L = events_4t_t_neut_1L[(lepton_indices_1L[:,0]==4)]
events_4t_t_neut_6_1L = events_4t_t_neut_1L[(lepton_indices_1L[:,0]==6)]

events_4t_r_neut_0_1L = events_4t_r_neut_1L[(lepton_indices_1L[:,0]==0)]
events_4t_r_neut_2_1L = events_4t_r_neut_1L[(lepton_indices_1L[:,0]==2)]
events_4t_r_neut_4_1L = events_4t_r_neut_1L[(lepton_indices_1L[:,0]==4)]
events_4t_r_neut_6_1L = events_4t_r_neut_1L[(lepton_indices_1L[:,0]==6)]

In [14]:
events_4t_truth_1L = ak.concatenate((events_4t_truth_0_1L, events_4t_truth_2_1L, events_4t_truth_4_1L, events_4t_truth_6_1L))

In [15]:
events_4t_reco_1L = ak.concatenate((events_4t_reco_0_1L, events_4t_reco_2_1L, events_4t_reco_4_1L, events_4t_reco_6_1L))

In [16]:
events_4t_t_neut_1L = ak.concatenate((events_4t_t_neut_0_1L, events_4t_t_neut_2_1L, events_4t_t_neut_4_1L, events_4t_t_neut_6_1L))
events_4t_r_neut_1L = ak.concatenate((events_4t_r_neut_0_1L, events_4t_r_neut_2_1L, events_4t_r_neut_4_1L, events_4t_r_neut_6_1L))

In [17]:
fm_events_4t_reco = events_4t_reco_1L[fm_mask_4t_1L]
fm_events_4t_truth = events_4t_truth_1L[fm_mask_4t_1L]
fm_events_4t_t_neut = events_4t_t_neut_1L[fm_mask_4t_1L]
fm_events_4t_r_neut = events_4t_r_neut_1L[fm_mask_4t_1L]

fm_truth_4t_1L = {}
fm_reco_4t_1L = {}
fm_t_neut_4t_1L = {}
fm_r_neut_4t_1L = {}

for field in fm_events_4t_truth.fields:
    fm_truth_4t_1L[f"{field}"] = fm_events_4t_truth[field]
for field in fm_events_4t_reco.fields:
    fm_reco_4t_1L[f"{field}"] = fm_events_4t_reco[field]
for field in fm_events_4t_t_neut.fields:
    fm_t_neut_4t_1L[f"{field}"] = fm_events_4t_t_neut[field]
for field in fm_events_4t_r_neut.fields:
    fm_r_neut_4t_1L[f"{field}"] = fm_events_4t_r_neut[field]

with  uproot.recreate(f"fm_4t_1L_02Dec.root") as file:
       file["Truth"] = fm_truth_4t_1L
       file["Reco"] = fm_reco_4t_1L
       file["Truth;Neutrino"] = fm_t_neut_4t_1L
       file["Reco;Neutrino"] = fm_r_neut_4t_1L

tm_events_4t_reco = events_4t_reco_1L[tm_mask_4t_1L]
tm_events_4t_truth = events_4t_truth_1L[tm_mask_4t_1L]
tm_events_4t_t_neut = events_4t_t_neut_1L[tm_mask_4t_1L]
tm_events_4t_r_neut = events_4t_r_neut_1L[tm_mask_4t_1L]

tm_truth_4t_1L = {}
tm_reco_4t_1L = {}
tm_t_neut_4t_1L = {}
tm_r_neut_4t_1L = {}

for field in tm_events_4t_truth.fields:
    tm_truth_4t_1L[f"{field}"] = tm_events_4t_truth[field]
for field in tm_events_4t_reco.fields:
    tm_reco_4t_1L[f"{field}"] = tm_events_4t_reco[field]
for field in tm_events_4t_t_neut.fields:
    tm_t_neut_4t_1L[f"{field}"] = tm_events_4t_t_neut[field]
for field in tm_events_4t_r_neut.fields:
    tm_r_neut_4t_1L[f"{field}"] = tm_events_4t_r_neut[field]

with  uproot.recreate(f"tm_4t_1L_02Dec.root") as file:
       file["Truth"] = tm_truth_4t_1L
       file["Reco"] = tm_reco_4t_1L
       file["Truth;Neutrino"] = tm_t_neut_4t_1L
       file["Reco;Neutrino"] = tm_r_neut_4t_1L

dm_events_4t_reco = events_4t_reco_1L[dm_mask_4t_1L]
dm_events_4t_truth = events_4t_truth_1L[dm_mask_4t_1L]
dm_events_4t_t_neut = events_4t_t_neut_1L[dm_mask_4t_1L]
dm_events_4t_r_neut = events_4t_r_neut_1L[dm_mask_4t_1L]

dm_truth_4t_1L = {}
dm_reco_4t_1L = {}
dm_t_neut_4t_1L = {}
dm_r_neut_4t_1L = {}

for field in dm_events_4t_truth.fields:
    dm_truth_4t_1L[f"{field}"] = dm_events_4t_truth[field]
for field in dm_events_4t_reco.fields:
    dm_reco_4t_1L[f"{field}"] = dm_events_4t_reco[field]
for field in dm_events_4t_t_neut.fields:
    dm_t_neut_4t_1L[f"{field}"] = dm_events_4t_t_neut[field]
for field in dm_events_4t_r_neut.fields:
    dm_r_neut_4t_1L[f"{field}"] = dm_events_4t_r_neut[field]

with  uproot.recreate(f"dm_4t_1L_02Dec.root") as file:
       file["Truth"] = dm_truth_4t_1L
       file["Reco"] = dm_reco_4t_1L
       file["Truth;Neutrino"] = dm_t_neut_4t_1L
       file["Reco;Neutrino"] = dm_r_neut_4t_1L

sm_events_4t_reco = events_4t_reco_1L[sm_mask_4t_1L]
sm_events_4t_truth = events_4t_truth_1L[sm_mask_4t_1L]
sm_events_4t_t_neut = events_4t_t_neut_1L[sm_mask_4t_1L]
sm_events_4t_r_neut = events_4t_r_neut_1L[sm_mask_4t_1L]

sm_truth_4t_1L = {}
sm_reco_4t_1L = {}
sm_t_neut_4t_1L = {}
sm_r_neut_4t_1L = {}

for field in sm_events_4t_truth.fields:
    sm_truth_4t_1L[f"{field}"] = sm_events_4t_truth[field]
for field in sm_events_4t_reco.fields:
    sm_reco_4t_1L[f"{field}"] = sm_events_4t_reco[field]
for field in sm_events_4t_t_neut.fields:
    sm_t_neut_4t_1L[f"{field}"] = sm_events_4t_t_neut[field]
for field in sm_events_4t_r_neut.fields:
    sm_r_neut_4t_1L[f"{field}"] = sm_events_4t_r_neut[field]

with  uproot.recreate(f"sm_4t_1L_02Dec.root") as file:
       file["Truth"] = sm_truth_4t_1L
       file["Reco"] = sm_reco_4t_1L
       file["Truth;Neutrino"] = sm_t_neut_4t_1L
       file["Reco;Neutrino"] = sm_r_neut_4t_1L

#### 2 Lepton Channel

In [5]:
matched_events_4t_2L, no_dupes_bool_2L = truth_matching(truth_4t_2L, truth_id_4t_2L, reco_4t_jets_2L, reco_4t_leptons_2L)

matched_4t_2L = ak.zip({'reco': matched_events_4t_2L['reco'], 'truth': matched_events_4t_2L['truth']})
events_4t_truth_2L = events_4t_truth_2L[no_dupes_bool_2L]
events_4t_reco_2L = events_4t_reco_2L[no_dupes_bool_2L]
lepton_indices_2L = ak.sort(ak.argsort(abs(events_4t_truth_2L['W_decay_id']), axis = 1, ascending=False)[:,:2], axis=1)

10
starting dm
starting axis=0 concat


In [6]:
sm_mask_4t_2L, dm_mask_4t_2L, tm_mask_4t_2L, fm_mask_4t_2L= top_identifier(matched_4t_2L, '4t', '2L', lepton_indices=lepton_indices_2L)

In [7]:
events_4t_truth_01_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_4t_truth_03_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_4t_truth_05_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==5)]
events_4t_truth_23_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]
events_4t_truth_25_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==5)]
events_4t_truth_45_2L = events_4t_truth_2L[(lepton_indices_2L[:,0]==4)&(lepton_indices_2L[:,1]==5)]

events_4t_reco_01_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_4t_reco_03_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_4t_reco_05_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==5)]
events_4t_reco_23_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]
events_4t_reco_25_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==5)]
events_4t_reco_45_2L = events_4t_reco_2L[(lepton_indices_2L[:,0]==4)&(lepton_indices_2L[:,1]==5)]


In [8]:
events_4t_truth_2L = ak.concatenate((events_4t_truth_01_2L, events_4t_truth_03_2L, events_4t_truth_05_2L, events_4t_truth_23_2L, events_4t_truth_25_2L, events_4t_truth_45_2L))

In [9]:
events_4t_reco_2L = ak.concatenate((events_4t_reco_01_2L, events_4t_reco_03_2L, events_4t_reco_05_2L, events_4t_reco_23_2L, events_4t_reco_25_2L, events_4t_reco_45_2L))

In [10]:
fm_events_4t_reco = events_4t_reco_2L[fm_mask_4t_2L]
fm_events_4t_truth = events_4t_truth_2L[fm_mask_4t_2L]

fm_truth_4t_2L = {}
fm_reco_4t_2L = {}

for field in fm_events_4t_truth.fields:
    fm_truth_4t_2L[f"{field}"] = fm_events_4t_truth[field]
for field in fm_events_4t_reco.fields:
    fm_reco_4t_2L[f"{field}"] = fm_events_4t_reco[field]

with  uproot.recreate(f"fm_4t_2L_02Dec.root") as file:
       file["Truth"] = fm_truth_4t_2L
       file["Reco"] = fm_reco_4t_2L

tm_events_4t_reco = events_4t_reco_2L[tm_mask_4t_2L]
tm_events_4t_truth = events_4t_truth_2L[tm_mask_4t_2L]

tm_truth_4t_2L = {}
tm_reco_4t_2L = {}

for field in tm_events_4t_truth.fields:
    tm_truth_4t_2L[f"{field}"] = tm_events_4t_truth[field]
for field in tm_events_4t_reco.fields:
    tm_reco_4t_2L[f"{field}"] = tm_events_4t_reco[field]

with  uproot.recreate(f"tm_4t_2L_02Dec.root") as file:
       file["Truth"] = tm_truth_4t_2L
       file["Reco"] = tm_reco_4t_2L

dm_events_4t_reco = events_4t_reco_2L[dm_mask_4t_2L]
dm_events_4t_truth = events_4t_truth_2L[dm_mask_4t_2L]

dm_truth_4t_2L = {}
dm_reco_4t_2L = {}

for field in dm_events_4t_truth.fields:
    dm_truth_4t_2L[f"{field}"] = dm_events_4t_truth[field]
for field in dm_events_4t_reco.fields:
    dm_reco_4t_2L[f"{field}"] = dm_events_4t_reco[field]

with  uproot.recreate(f"dm_4t_2L_02Dec.root") as file:
       file["Truth"] = dm_truth_4t_2L
       file["Reco"] = dm_reco_4t_2L

sm_events_4t_reco = events_4t_reco_2L[sm_mask_4t_2L]
sm_events_4t_truth = events_4t_truth_2L[sm_mask_4t_2L]

sm_truth_4t_2L = {}
sm_reco_4t_2L = {}

for field in sm_events_4t_truth.fields:
    sm_truth_4t_2L[f"{field}"] = sm_events_4t_truth[field]
for field in sm_events_4t_reco.fields:
    sm_reco_4t_2L[f"{field}"] = sm_events_4t_reco[field]

with  uproot.recreate(f"sm_4t_2L_02Dec.root") as file:
       file["Truth"] = sm_truth_4t_2L
       file["Reco"] = sm_reco_4t_2L

## 3 top Matching

### 3tW Data

In [9]:
tree3W = uproot.open(directory + "3tW_combined_parsed_0212.root")
tree3W_truth = tree3W['Truth;1']
tree3W_reco = tree3W['Reco;1']

events_3tW_truth = tree3W_truth.arrays(['b_id'])

In [10]:
len(events_3tW_truth['b_id'])

6912317

In [3]:
tree3W = uproot.open(directory + "3tW_combined_parsed_0212.root")
tree3W_truth = tree3W['Truth;1']
tree3W_reco = tree3W['Reco;1']

events_3tW_truth = tree3W_truth.arrays(['b_id', 'b_pt', 'b_eta', 'b_phi', 'b_e', 'b_mass', 'W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_3tW_t_neut = tree3W_truth.arrays(['W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_3tW_partonic = tree3W_truth.arrays(['top_pt', 'top_eta', 'top_phi', 'top_e', 'top_mass', 'top_id', 'W_pt', 'W_eta', 'W_phi', 'W_e', 'W_mass', 'W_id'])
events_3tW_reco = tree3W_reco.arrays(['jet_pt', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_btag', 'el_pt', 'el_eta', 'el_phi', 'el_charge', 'mu_pt', 'mu_eta', 'mu_phi', 'mu_charge', 'met_met', 'met_eta', 'met_phi'])
events_3tW_r_neut = tree3W_reco.arrays(['met_met', 'met_eta', 'met_phi'])


events_3tW_reco['el_mass'] = 0.511e-3 * np.ones_like(events_3tW_reco['el_pt'])
events_3tW_reco['mu_mass'] = 0.1057 * np.ones_like(events_3tW_reco['mu_pt'])
events_3tW_reco['jet_e'] = np.sqrt(events_3tW_reco['jet_mass']**2+(events_3tW_reco['jet_pt']**2)*(np.cosh(events_3tW_reco['jet_eta']))**2)
events_3tW_reco['el_e'] = np.sqrt(events_3tW_reco['el_mass']**2+(events_3tW_reco['el_pt']**2)*(np.cosh(events_3tW_reco['el_eta']))**2)
events_3tW_reco['mu_e'] = np.sqrt(events_3tW_reco['mu_mass']**2+(events_3tW_reco['mu_pt']**2)*(np.cosh(events_3tW_reco['mu_eta']))**2)
events_3tW_r_neut['met_e'] = (events_3tW_r_neut['met_met'])*(np.cosh(events_3tW_r_neut['met_eta']))

good_events_3tW = ak.num(events_3tW_truth['b_id']) == 3

invalid_events_3tW_truth = events_3tW_truth[~good_events_3tW]
invalid_events_3tW_truth['b_id'], invalid_events_3tW_truth['b_pt'], invalid_events_3tW_truth['b_eta'] = invalid_events_3tW_truth['b_id'][:,1:], invalid_events_3tW_truth['b_pt'][:,1:], invalid_events_3tW_truth['b_eta'][:,1:]
invalid_events_3tW_truth['b_phi'], invalid_events_3tW_truth['b_e'], invalid_events_3tW_truth['b_mass'] = invalid_events_3tW_truth['b_phi'][:,1:], invalid_events_3tW_truth['b_e'][:,1:], invalid_events_3tW_truth['b_mass'][:,1:]

events_3tW_truth = ak.concatenate((events_3tW_truth[good_events_3tW], invalid_events_3tW_truth))
events_3tW_reco = ak.concatenate((events_3tW_reco[good_events_3tW], events_3tW_reco[~good_events_3tW]))
events_3tW_partonic = ak.concatenate((events_3tW_partonic[good_events_3tW], events_3tW_partonic[~good_events_3tW]))
events_3tW_t_neut = ak.concatenate((events_3tW_t_neut[good_events_3tW], events_3tW_t_neut[~good_events_3tW]))
events_3tW_r_neut = ak.concatenate((events_3tW_r_neut[good_events_3tW], events_3tW_r_neut[~good_events_3tW]))

x = events_3tW_truth['W_decay_id'][(events_3tW_truth['W_decay_id']==15)|(events_3tW_truth['W_decay_id']==-15)]
events_3tW_reco = events_3tW_reco[ak.num(x)==0]
events_3tW_partonic = events_3tW_partonic[ak.num(x)==0]
events_3tW_truth = events_3tW_truth[ak.num(x)==0]
events_3tW_t_neut = events_3tW_t_neut[ak.num(x)==0]
events_3tW_r_neut = events_3tW_r_neut[ak.num(x)==0]

neutrino_remover = (abs(events_3tW_truth['W_decay_id']) != 12)&(abs(events_3tW_truth['W_decay_id']) != 14)&(abs(events_3tW_truth['W_decay_id']) != 16)&(abs(events_3tW_truth['W_decay_id']) != 18)
events_3tW_t_neut['W_decay_id'] = events_3tW_t_neut['W_decay_id'][~neutrino_remover]
events_3tW_t_neut['W_decay_pt'] = events_3tW_t_neut['W_decay_pt'][~neutrino_remover]
events_3tW_t_neut['W_decay_eta'] = events_3tW_t_neut['W_decay_eta'][~neutrino_remover]
events_3tW_t_neut['W_decay_phi'] = events_3tW_t_neut['W_decay_phi'][~neutrino_remover]
events_3tW_t_neut['W_decay_e'] = events_3tW_t_neut['W_decay_e'][~neutrino_remover]
events_3tW_t_neut['W_decay_mass'] = events_3tW_t_neut['W_decay_mass'][~neutrino_remover]

events_3tW_truth['W_decay_id'] = events_3tW_truth['W_decay_id'][neutrino_remover]
events_3tW_truth['W_decay_pt'] = events_3tW_truth['W_decay_pt'][neutrino_remover]
events_3tW_truth['W_decay_eta'] = events_3tW_truth['W_decay_eta'][neutrino_remover]
events_3tW_truth['W_decay_phi'] = events_3tW_truth['W_decay_phi'][neutrino_remover]
events_3tW_truth['W_decay_e'] = events_3tW_truth['W_decay_e'][neutrino_remover]
events_3tW_truth['W_decay_mass'] = events_3tW_truth['W_decay_mass'][neutrino_remover]

jet_pt_3tW = events_3tW_reco['jet_pt'][(events_3tW_reco['jet_eta']<2.5) & (events_3tW_reco['jet_eta']>-2.5)]
jet_eta_3tW = events_3tW_reco['jet_eta'][(events_3tW_reco['jet_eta']<2.5) & (events_3tW_reco['jet_eta']>-2.5)]
jet_phi_3tW = events_3tW_reco['jet_phi'][(events_3tW_reco['jet_eta']<2.5) & (events_3tW_reco['jet_eta']>-2.5)]
jet_mass_3tW = events_3tW_reco['jet_mass'][(events_3tW_reco['jet_eta']<2.5) & (events_3tW_reco['jet_eta']>-2.5)]
jet_e_3tW = events_3tW_reco['jet_e'][(events_3tW_reco['jet_eta']<2.5) & (events_3tW_reco['jet_eta']>-2.5)]

electron_pt_3tW = events_3tW_reco['el_pt'][(events_3tW_reco['el_pt'] > 15) & (events_3tW_reco['el_eta'] < 2.47) & (events_3tW_reco['el_eta'] > -2.47)]
electron_eta_3tW = events_3tW_reco['el_eta'][(events_3tW_reco['el_pt'] > 15) & (events_3tW_reco['el_eta'] < 2.47) & (events_3tW_reco['el_eta'] > -2.47)]
electron_phi_3tW = events_3tW_reco['el_phi'][(events_3tW_reco['el_pt'] > 15) & (events_3tW_reco['el_eta'] < 2.47) & (events_3tW_reco['el_eta'] > -2.47)]
electron_mass_3tW = events_3tW_reco['el_mass'][(events_3tW_reco['el_pt'] > 15) & (events_3tW_reco['el_eta'] < 2.47) & (events_3tW_reco['el_eta'] > -2.47)]
electron_e_3tW = events_3tW_reco['el_e'][(events_3tW_reco['el_pt'] > 15) & (events_3tW_reco['el_eta'] < 2.47) & (events_3tW_reco['el_eta'] > -2.47)]

muon_pt_3tW = events_3tW_reco['mu_pt'][(events_3tW_reco['mu_pt']>15) & (events_3tW_reco['mu_eta']<2.5) & (events_3tW_reco['mu_eta']>-2.5)]
muon_eta_3tW = events_3tW_reco['mu_eta'][(events_3tW_reco['mu_pt']>15) & (events_3tW_reco['mu_eta']<2.5) & (events_3tW_reco['mu_eta']>-2.5)]
muon_phi_3tW = events_3tW_reco['mu_phi'][(events_3tW_reco['mu_pt']>15) & (events_3tW_reco['mu_eta']<2.5) & (events_3tW_reco['mu_eta']>-2.5)]
muon_mass_3tW = events_3tW_reco['mu_mass'][(events_3tW_reco['mu_pt']>15) & (events_3tW_reco['mu_eta']<2.5) & (events_3tW_reco['mu_eta']>-2.5)]
muon_e_3tW = events_3tW_reco['mu_e'][(events_3tW_reco['mu_pt']>15) & (events_3tW_reco['mu_eta']<2.5) & (events_3tW_reco['mu_eta']>-2.5)]

lepton_pt_3tW = ak.concatenate((muon_pt_3tW,electron_pt_3tW), axis=1)
lepton_eta_3tW = ak.concatenate((muon_eta_3tW,electron_eta_3tW), axis=1)
lepton_phi_3tW = ak.concatenate((muon_phi_3tW,electron_phi_3tW), axis=1)
lepton_mass_3tW = ak.concatenate((muon_mass_3tW,electron_mass_3tW), axis=1)
lepton_e_3tW = ak.concatenate((muon_e_3tW,electron_e_3tW), axis=1)

reco_pt_3tW = ak.concatenate((jet_pt_3tW,lepton_pt_3tW), axis=1)
reco_eta_3tW = ak.concatenate((jet_eta_3tW,lepton_eta_3tW), axis=1)
reco_phi_3tW = ak.concatenate((jet_phi_3tW,lepton_phi_3tW), axis=1)
reco_mass_3tW = ak.concatenate((jet_mass_3tW,lepton_mass_3tW), axis=1)
reco_e_3tW = ak.concatenate((jet_e_3tW,lepton_e_3tW), axis=1)

b_pt_3tW = events_3tW_truth['b_pt']
b_eta_3tW = events_3tW_truth['b_eta']
b_phi_3tW = events_3tW_truth['b_phi']
b_mass_3tW = events_3tW_truth['b_mass']
b_e_3tW = events_3tW_truth['b_e']
b_id_3tW = events_3tW_truth['b_id']

w_decay_pt_3tW = events_3tW_truth['W_decay_pt']
w_decay_eta_3tW = events_3tW_truth['W_decay_eta']
w_decay_phi_3tW = events_3tW_truth['W_decay_phi']
w_decay_mass_3tW = events_3tW_truth['W_decay_mass']
w_decay_e_3tW = events_3tW_truth['W_decay_e']
w_decay_id_3tW = events_3tW_truth['W_decay_id']

truth_pt_3tW = ak.concatenate((b_pt_3tW,w_decay_pt_3tW), axis=1)
truth_eta_3tW = ak.concatenate((b_eta_3tW,w_decay_eta_3tW), axis=1)
truth_phi_3tW = ak.concatenate((b_phi_3tW,w_decay_phi_3tW), axis=1)
truth_mass_3tW = ak.concatenate((b_mass_3tW,w_decay_mass_3tW), axis=1)
truth_e_3tW = ak.concatenate((b_e_3tW,w_decay_e_3tW), axis=1)
truth_id_3tW = ak.concatenate((b_id_3tW,w_decay_id_3tW), axis=1)

reco_3tW = vector.zip({'pt':reco_pt_3tW,'eta':reco_eta_3tW,'phi':reco_phi_3tW,'energy':reco_e_3tW})
truth_3tW = vector.zip({'pt':truth_pt_3tW,'eta':truth_eta_3tW,'phi':truth_phi_3tW,'energy':truth_e_3tW})

reco_3tW_leptons = vector.zip({'pt':lepton_pt_3tW,'eta':lepton_eta_3tW,'phi':lepton_phi_3tW,'energy':lepton_e_3tW})
reco_3tW_jets = vector.zip({'pt':jet_pt_3tW,'eta':jet_eta_3tW,'phi':jet_phi_3tW,'energy':jet_e_3tW})

In [ ]:
truth_3tW = truth_3tW[(ak.num(reco_3tW)!=0)]
events_3tW_truth = events_3tW_truth[(ak.num(reco_3tW)!=0)]
events_3tW_reco = events_3tW_truth[(ak.num(reco_3tW)!=0)]
events_3tW_t_neut = events_3tW_truth[(ak.num(reco_3tW)!=0)]
reco_3tW_leptons = reco_3tW_leptons[(ak.num(reco_3tW)!=0)]
reco_3tW_jets = reco_3tW_jets[(ak.num(reco_3tW)!=0)]
reco_3tW = reco_3tW[(ak.num(reco_3tW)!=0)]

### 3tj Data

In [4]:
tree3j = uproot.open(directory + "3tj_combined_parsed_0212.root")
tree3j_truth = tree3j['Truth;1']
tree3j_reco = tree3j['Reco;1']

events_3tj_truth = tree3j_truth.arrays(['b_id', 'b_pt', 'b_eta', 'b_phi', 'b_e', 'b_mass', 'W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_3tj_t_neut = tree3j_truth.arrays(['W_decay_id', 'W_decay_pt', 'W_decay_eta', 'W_decay_phi', 'W_decay_e', 'W_decay_mass'])
events_3tj_partonic = tree3j_truth.arrays(['top_pt', 'top_eta', 'top_phi', 'top_e', 'top_mass', 'top_id', 'W_pt', 'W_eta', 'W_phi', 'W_e', 'W_mass', 'W_id'])
events_3tj_reco = tree3j_reco.arrays(['jet_pt', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_btag', 'el_pt', 'el_eta', 'el_phi', 'el_charge', 'mu_pt', 'mu_eta', 'mu_phi', 'mu_charge', 'met_met', 'met_eta', 'met_phi'])
events_3tj_r_neut = tree3j_reco.arrays(['met_met', 'met_eta', 'met_phi'])

events_3tj_reco['el_mass'] = 0.511e-3 * np.ones_like(events_3tj_reco['el_pt'])
events_3tj_reco['mu_mass'] = 0.1057 * np.ones_like(events_3tj_reco['mu_pt'])
events_3tj_reco['jet_e'] = np.sqrt(events_3tj_reco['jet_mass']**2+(events_3tj_reco['jet_pt']**2)*(np.cosh(events_3tj_reco['jet_eta']))**2)
events_3tj_reco['el_e'] = np.sqrt(events_3tj_reco['el_mass']**2+(events_3tj_reco['el_pt']**2)*(np.cosh(events_3tj_reco['el_eta']))**2)
events_3tj_reco['mu_e'] = np.sqrt(events_3tj_reco['mu_mass']**2+(events_3tj_reco['mu_pt']**2)*(np.cosh(events_3tj_reco['mu_eta']))**2)
events_3tj_r_neut['met_e'] = (events_3tj_r_neut['met_met'])*(np.cosh(events_3tj_r_neut['met_eta']))

good_events_3tj = ak.num(events_3tj_truth['b_id']) == 3

invalid_events_3tj_truth = events_3tj_truth[~good_events_3tj]
invalid_events_3tj_truth['b_id'], invalid_events_3tj_truth['b_pt'], invalid_events_3tj_truth['b_eta'] = invalid_events_3tj_truth['b_id'][:,1:], invalid_events_3tj_truth['b_pt'][:,1:], invalid_events_3tj_truth['b_eta'][:,1:]
invalid_events_3tj_truth['b_phi'], invalid_events_3tj_truth['b_e'], invalid_events_3tj_truth['b_mass'] = invalid_events_3tj_truth['b_phi'][:,1:], invalid_events_3tj_truth['b_e'][:,1:], invalid_events_3tj_truth['b_mass'][:,1:]

events_3tj_truth = ak.concatenate((events_3tj_truth[good_events_3tj], invalid_events_3tj_truth))
events_3tj_reco = ak.concatenate((events_3tj_reco[good_events_3tj], events_3tj_reco[~good_events_3tj]))
events_3tj_partonic = ak.concatenate((events_3tj_partonic[good_events_3tj], events_3tj_partonic[~good_events_3tj]))
events_3tj_t_neut = ak.concatenate((events_3tj_t_neut[good_events_3tj], events_3tj_t_neut[~good_events_3tj]))
events_3tj_r_neut = ak.concatenate((events_3tj_r_neut[good_events_3tj], events_3tj_r_neut[~good_events_3tj]))

x = events_3tj_truth['W_decay_id'][(events_3tj_truth['W_decay_id']==15)|(events_3tj_truth['W_decay_id']==-15)]
events_3tj_reco = events_3tj_reco[ak.num(x)==0]
events_3tj_partonic = events_3tj_partonic[ak.num(x)==0]
events_3tj_truth = events_3tj_truth[ak.num(x)==0]
events_3tj_t_neut = events_3tj_t_neut[ak.num(x)==0]
events_3tj_r_neut = events_3tj_r_neut[ak.num(x)==0]

neutrino_remover = (abs(events_3tj_truth['W_decay_id']) != 12)&(abs(events_3tj_truth['W_decay_id']) != 14)&(abs(events_3tj_truth['W_decay_id']) != 16)&(abs(events_3tj_truth['W_decay_id']) != 18)
events_3tj_t_neut['W_decay_id'] = events_3tj_t_neut['W_decay_id'][~neutrino_remover]
events_3tj_t_neut['W_decay_pt'] = events_3tj_t_neut['W_decay_pt'][~neutrino_remover]
events_3tj_t_neut['W_decay_eta'] = events_3tj_t_neut['W_decay_eta'][~neutrino_remover]
events_3tj_t_neut['W_decay_phi'] = events_3tj_t_neut['W_decay_phi'][~neutrino_remover]
events_3tj_t_neut['W_decay_e'] = events_3tj_t_neut['W_decay_e'][~neutrino_remover]
events_3tj_t_neut['W_decay_mass'] = events_3tj_t_neut['W_decay_mass'][~neutrino_remover]

events_3tj_truth['W_decay_id'] = events_3tj_truth['W_decay_id'][neutrino_remover]
events_3tj_truth['W_decay_pt'] = events_3tj_truth['W_decay_pt'][neutrino_remover]
events_3tj_truth['W_decay_eta'] = events_3tj_truth['W_decay_eta'][neutrino_remover]
events_3tj_truth['W_decay_phi'] = events_3tj_truth['W_decay_phi'][neutrino_remover]
events_3tj_truth['W_decay_e'] = events_3tj_truth['W_decay_e'][neutrino_remover]
events_3tj_truth['W_decay_mass'] = events_3tj_truth['W_decay_mass'][neutrino_remover]

jet_pt_3tj = events_3tj_reco['jet_pt'][(events_3tj_reco['jet_eta']<2.5) & (events_3tj_reco['jet_eta']>-2.5)]
jet_eta_3tj = events_3tj_reco['jet_eta'][(events_3tj_reco['jet_eta']<2.5) & (events_3tj_reco['jet_eta']>-2.5)]
jet_phi_3tj = events_3tj_reco['jet_phi'][(events_3tj_reco['jet_eta']<2.5) & (events_3tj_reco['jet_eta']>-2.5)]
jet_mass_3tj = events_3tj_reco['jet_mass'][(events_3tj_reco['jet_eta']<2.5) & (events_3tj_reco['jet_eta']>-2.5)]
jet_e_3tj = events_3tj_reco['jet_e'][(events_3tj_reco['jet_eta']<2.5) & (events_3tj_reco['jet_eta']>-2.5)]

electron_pt_3tj = events_3tj_reco['el_pt'][(events_3tj_reco['el_pt'] > 15) & (events_3tj_reco['el_eta'] < 2.47) & (events_3tj_reco['el_eta'] > -2.47)]
electron_eta_3tj = events_3tj_reco['el_eta'][(events_3tj_reco['el_pt'] > 15) & (events_3tj_reco['el_eta'] < 2.47) & (events_3tj_reco['el_eta'] > -2.47)]
electron_phi_3tj = events_3tj_reco['el_phi'][(events_3tj_reco['el_pt'] > 15) & (events_3tj_reco['el_eta'] < 2.47) & (events_3tj_reco['el_eta'] > -2.47)]
electron_mass_3tj = events_3tj_reco['el_mass'][(events_3tj_reco['el_pt'] > 15) & (events_3tj_reco['el_eta'] < 2.47) & (events_3tj_reco['el_eta'] > -2.47)]
electron_e_3tj = events_3tj_reco['el_e'][(events_3tj_reco['el_pt'] > 15) & (events_3tj_reco['el_eta'] < 2.47) & (events_3tj_reco['el_eta'] > -2.47)]

muon_pt_3tj = events_3tj_reco['mu_pt'][(events_3tj_reco['mu_pt']>15) & (events_3tj_reco['mu_eta']<2.5) & (events_3tj_reco['mu_eta']>-2.5)]
muon_eta_3tj = events_3tj_reco['mu_eta'][(events_3tj_reco['mu_pt']>15) & (events_3tj_reco['mu_eta']<2.5) & (events_3tj_reco['mu_eta']>-2.5)]
muon_phi_3tj = events_3tj_reco['mu_phi'][(events_3tj_reco['mu_pt']>15) & (events_3tj_reco['mu_eta']<2.5) & (events_3tj_reco['mu_eta']>-2.5)]
muon_mass_3tj = events_3tj_reco['mu_mass'][(events_3tj_reco['mu_pt']>15) & (events_3tj_reco['mu_eta']<2.5) & (events_3tj_reco['mu_eta']>-2.5)]
muon_e_3tj = events_3tj_reco['mu_e'][(events_3tj_reco['mu_pt']>15) & (events_3tj_reco['mu_eta']<2.5) & (events_3tj_reco['mu_eta']>-2.5)]

lepton_pt_3tj = ak.concatenate((muon_pt_3tj,electron_pt_3tj), axis=1)
lepton_eta_3tj = ak.concatenate((muon_eta_3tj,electron_eta_3tj), axis=1)
lepton_phi_3tj = ak.concatenate((muon_phi_3tj,electron_phi_3tj), axis=1)
lepton_mass_3tj = ak.concatenate((muon_mass_3tj,electron_mass_3tj), axis=1)
lepton_e_3tj = ak.concatenate((muon_e_3tj,electron_e_3tj), axis=1)

reco_pt_3tj = ak.concatenate((jet_pt_3tj,lepton_pt_3tj), axis=1)
reco_eta_3tj = ak.concatenate((jet_eta_3tj,lepton_eta_3tj), axis=1)
reco_phi_3tj = ak.concatenate((jet_phi_3tj,lepton_phi_3tj), axis=1)
reco_mass_3tj = ak.concatenate((jet_mass_3tj,lepton_mass_3tj), axis=1)
reco_e_3tj = ak.concatenate((jet_e_3tj,lepton_e_3tj), axis=1)

b_pt_3tj = events_3tj_truth['b_pt']
b_eta_3tj = events_3tj_truth['b_eta']
b_phi_3tj = events_3tj_truth['b_phi']
b_mass_3tj = events_3tj_truth['b_mass']
b_e_3tj = events_3tj_truth['b_e']
b_id_3tj = events_3tj_truth['b_id']

w_decay_pt_3tj = events_3tj_truth['W_decay_pt']
w_decay_eta_3tj = events_3tj_truth['W_decay_eta']
w_decay_phi_3tj = events_3tj_truth['W_decay_phi']
w_decay_mass_3tj = events_3tj_truth['W_decay_mass']
w_decay_e_3tj = events_3tj_truth['W_decay_e']
w_decay_id_3tj = events_3tj_truth['W_decay_id']

truth_pt_3tj = ak.concatenate((b_pt_3tj,w_decay_pt_3tj), axis=1)
truth_eta_3tj = ak.concatenate((b_eta_3tj,w_decay_eta_3tj), axis=1)
truth_phi_3tj = ak.concatenate((b_phi_3tj,w_decay_phi_3tj), axis=1)
truth_mass_3tj = ak.concatenate((b_mass_3tj,w_decay_mass_3tj), axis=1)
truth_e_3tj = ak.concatenate((b_e_3tj,w_decay_e_3tj), axis=1)
truth_id_3tj = ak.concatenate((b_id_3tj,w_decay_id_3tj), axis=1)

reco_3tj = vector.zip({'pt':reco_pt_3tj,'eta':reco_eta_3tj,'phi':reco_phi_3tj,'energy':reco_e_3tj})
truth_3tj = vector.zip({'pt':truth_pt_3tj,'eta':truth_eta_3tj,'phi':truth_phi_3tj,'energy':truth_e_3tj})

reco_3tj_leptons = vector.zip({'pt':lepton_pt_3tj,'eta':lepton_eta_3tj,'phi':lepton_phi_3tj,'energy':lepton_e_3tj})
reco_3tj_jets = vector.zip({'pt':jet_pt_3tj,'eta':jet_eta_3tj,'phi':jet_phi_3tj,'energy':jet_e_3tj})

### Channel Splitting for 3 tops

In [7]:
events_3tW_truth = events_3tW_truth[(ak.num(reco_3tW)!=0)]

In [4]:
truth_lepton_count_3tW = ak.num(events_3tW_truth['W_decay_id'][(abs(events_3tW_truth['W_decay_id'])>10)&(abs(events_3tW_truth['W_decay_id'])<19)])
reco_lepton_count_3tW = ak.num(reco_3tW_leptons)

mask_0L_3tW = (reco_lepton_count_3tW == 0) & (truth_lepton_count_3tW == 0)
mask_1L_3tW = (reco_lepton_count_3tW == 1) & (truth_lepton_count_3tW == 1)
mask_2L_3tW = (reco_lepton_count_3tW == 2) & (truth_lepton_count_3tW == 2)
mask_3L_3tW = (reco_lepton_count_3tW == 3) & (truth_lepton_count_3tW == 3)

events_3tW_truth_0L, events_3tW_reco_0L = events_3tW_truth[mask_0L_3tW], events_3tW_reco[mask_0L_3tW]
events_3tW_truth_1L, events_3tW_reco_1L = events_3tW_truth[mask_1L_3tW], events_3tW_reco[mask_1L_3tW]
events_3tW_t_neut_1L, events_3tW_r_neut_1L = events_3tW_t_neut[mask_1L_3tW], events_3tW_r_neut[mask_1L_3tW]
events_3tW_truth_2L, events_3tW_reco_2L = events_3tW_truth[mask_2L_3tW], events_3tW_reco[mask_2L_3tW]
events_3tW_truth_3L, events_3tW_reco_3L = events_3tW_truth[mask_3L_3tW], events_3tW_reco[mask_3L_3tW]

truth_3tW_0L, truth_id_3tW_0L = truth_3tW[mask_0L_3tW], truth_id_3tW[mask_0L_3tW]
truth_3tW_1L, truth_id_3tW_1L = truth_3tW[mask_1L_3tW], truth_id_3tW[mask_1L_3tW]
truth_3tW_2L, truth_id_3tW_2L = truth_3tW[mask_2L_3tW], truth_id_3tW[mask_2L_3tW]
truth_3tW_3L, truth_id_3tW_3L = truth_3tW[mask_3L_3tW], truth_id_3tW[mask_3L_3tW]

reco_3tW_0L, reco_3tW_leptons_0L, reco_3tW_jets_0L = reco_3tW[mask_0L_3tW], reco_3tW_leptons[mask_0L_3tW], reco_3tW_jets[mask_0L_3tW]
reco_3tW_1L, reco_3tW_leptons_1L, reco_3tW_jets_1L = reco_3tW[mask_1L_3tW], reco_3tW_leptons[mask_1L_3tW], reco_3tW_jets[mask_1L_3tW]
reco_3tW_2L, reco_3tW_leptons_2L, reco_3tW_jets_2L = reco_3tW[mask_2L_3tW], reco_3tW_leptons[mask_2L_3tW], reco_3tW_jets[mask_2L_3tW]
reco_3tW_3L, reco_3tW_leptons_3L, reco_3tW_jets_3L = reco_3tW[mask_3L_3tW], reco_3tW_leptons[mask_3L_3tW], reco_3tW_jets[mask_3L_3tW]

In [5]:
truth_lepton_count_3tj = ak.num(events_3tj_truth['W_decay_id'][(abs(events_3tj_truth['W_decay_id'])>10)&(abs(events_3tj_truth['W_decay_id'])<19)])
reco_lepton_count_3tj = ak.num(reco_3tj_leptons)

mask_0L_3tj = (truth_lepton_count_3tj == 0) & (reco_lepton_count_3tj == 0)
mask_1L_3tj = (truth_lepton_count_3tj == 1) & (reco_lepton_count_3tj == 1)
mask_2L_3tj = (truth_lepton_count_3tj == 2) & (reco_lepton_count_3tj == 2)
mask_3L_3tj = (truth_lepton_count_3tj == 3) & (reco_lepton_count_3tj == 3)

events_3tj_truth_0L, events_3tj_reco_0L = events_3tj_truth[mask_0L_3tj], events_3tj_reco[mask_0L_3tj]
events_3tj_truth_1L, events_3tj_reco_1L = events_3tj_truth[mask_1L_3tj], events_3tj_reco[mask_1L_3tj]
events_3tj_t_neut_1L, events_3tj_r_neut_1L = events_3tj_t_neut[mask_1L_3tj], events_3tj_r_neut[mask_1L_3tj]
events_3tj_truth_2L, events_3tj_reco_2L = events_3tj_truth[mask_2L_3tj], events_3tj_reco[mask_2L_3tj]
events_3tj_truth_3L, events_3tj_reco_3L = events_3tj_truth[mask_3L_3tj], events_3tj_reco[mask_3L_3tj]

truth_3tj_0L, truth_id_3tj_0L = truth_3tj[mask_0L_3tj], truth_id_3tj[mask_0L_3tj]
truth_3tj_1L, truth_id_3tj_1L = truth_3tj[mask_1L_3tj], truth_id_3tj[mask_1L_3tj]
truth_3tj_2L, truth_id_3tj_2L = truth_3tj[mask_2L_3tj], truth_id_3tj[mask_2L_3tj]
truth_3tj_3L, truth_id_3tj_3L = truth_3tj[mask_3L_3tj], truth_id_3tj[mask_3L_3tj]

reco_3tj_0L, reco_3tj_leptons_0L, reco_3tj_jets_0L = reco_3tj[mask_0L_3tj], reco_3tj_leptons[mask_0L_3tj], reco_3tj_jets[mask_0L_3tj]
reco_3tj_1L, reco_3tj_leptons_1L, reco_3tj_jets_1L = reco_3tj[mask_1L_3tj], reco_3tj_leptons[mask_1L_3tj], reco_3tj_jets[mask_1L_3tj]
reco_3tj_2L, reco_3tj_leptons_2L, reco_3tj_jets_2L = reco_3tj[mask_2L_3tj], reco_3tj_leptons[mask_2L_3tj], reco_3tj_jets[mask_2L_3tj]
reco_3tj_3L, reco_3tj_leptons_3L, reco_3tj_jets_3L = reco_3tj[mask_3L_3tj], reco_3tj_leptons[mask_3L_3tj], reco_3tj_jets[mask_3L_3tj]

### Matching + Writing

#### 0 Lepton Channel

In [5]:
matched_events_3tW_0L, no_dupes_bool_0L = truth_matching(truth_3tW_0L, truth_id_3tW_0L, reco_3tW_jets_0L, reco_3tW_leptons_0L)
matched_3tW_0L = ak.zip({'reco': matched_events_3tW_0L['reco'], 'truth': matched_events_3tW_0L['truth']})
sm_mask_3tW_0L, dm_mask_3tW_0L, fm_mask_3tW_0L= top_identifier(matched_3tW_0L, '3tW', '0L')

11
starting dm
starting axis=0 concat


In [6]:
fm_events_3tW_reco = events_3tW_reco_0L[no_dupes_bool_0L][fm_mask_3tW_0L]
fm_events_3tW_truth = events_3tW_truth_0L[no_dupes_bool_0L][fm_mask_3tW_0L]

fm_truth_3tW_0L = {}
fm_reco_3tW_0L = {}

for field in fm_events_3tW_truth.fields:
    fm_truth_3tW_0L[f"{field}"] = fm_events_3tW_truth[field]
for field in fm_events_3tW_reco.fields:
    fm_reco_3tW_0L[f"{field}"] = fm_events_3tW_reco[field]

with  uproot.recreate(f"fm_3tW_0L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tW_0L
       file["Reco"] = fm_reco_3tW_0L

dm_events_3tW_reco = events_3tW_reco_0L[no_dupes_bool_0L][dm_mask_3tW_0L]
dm_events_3tW_truth = events_3tW_truth_0L[no_dupes_bool_0L][dm_mask_3tW_0L]

dm_truth_3tW_0L = {}
dm_reco_3tW_0L = {}

for field in dm_events_3tW_truth.fields:
    dm_truth_3tW_0L[f"{field}"] = dm_events_3tW_truth[field]
for field in dm_events_3tW_reco.fields:
    dm_reco_3tW_0L[f"{field}"] = dm_events_3tW_reco[field]

with  uproot.recreate(f"dm_3tW_0L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tW_0L
       file["Reco"] = dm_reco_3tW_0L

sm_events_3tW_reco = events_3tW_reco_0L[no_dupes_bool_0L][sm_mask_3tW_0L]
sm_events_3tW_truth = events_3tW_truth_0L[no_dupes_bool_0L][sm_mask_3tW_0L]

sm_truth_3tW_0L = {}
sm_reco_3tW_0L = {}

for field in sm_events_3tW_truth.fields:
    sm_truth_3tW_0L[f"{field}"] = sm_events_3tW_truth[field]
for field in sm_events_3tW_reco.fields:
    sm_reco_3tW_0L[f"{field}"] = sm_events_3tW_reco[field]

with  uproot.recreate(f"sm_3tW_0L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tW_0L
       file["Reco"] = sm_reco_3tW_0L

In [6]:
matched_events_3tj_0L, no_dupes_bool_0L = truth_matching(truth_3tj_0L, truth_id_3tj_0L, reco_3tj_jets_0L, reco_3tj_leptons_0L)
matched_3tj_0L = ak.zip({'reco': matched_events_3tj_0L['reco'], 'truth': matched_events_3tj_0L['truth']})
sm_mask_3tj_0L, dm_mask_3tj_0L, fm_mask_3tj_0L= top_identifier(matched_3tj_0L, '3tj', '0L')

9
starting dm
starting axis=0 concat


In [7]:
fm_events_3tj_reco = events_3tj_reco_0L[no_dupes_bool_0L][fm_mask_3tj_0L]
fm_events_3tj_truth = events_3tj_truth_0L[no_dupes_bool_0L][fm_mask_3tj_0L]

fm_truth_3tj_0L = {}
fm_reco_3tj_0L = {}

for field in fm_events_3tj_truth.fields:
    fm_truth_3tj_0L[f"{field}"] = fm_events_3tj_truth[field]
for field in fm_events_3tj_reco.fields:
    fm_reco_3tj_0L[f"{field}"] = fm_events_3tj_reco[field]

with  uproot.recreate(f"fm_3tj_0L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tj_0L
       file["Reco"] = fm_reco_3tj_0L

dm_events_3tj_reco = events_3tj_reco_0L[no_dupes_bool_0L][dm_mask_3tj_0L]
dm_events_3tj_truth = events_3tj_truth_0L[no_dupes_bool_0L][dm_mask_3tj_0L]

dm_truth_3tj_0L = {}
dm_reco_3tj_0L = {}

for field in dm_events_3tj_truth.fields:
    dm_truth_3tj_0L[f"{field}"] = dm_events_3tj_truth[field]
for field in dm_events_3tj_reco.fields:
    dm_reco_3tj_0L[f"{field}"] = dm_events_3tj_reco[field]

with  uproot.recreate(f"dm_3tj_0L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tj_0L
       file["Reco"] = dm_reco_3tj_0L

sm_events_3tj_reco = events_3tj_reco_0L[no_dupes_bool_0L][sm_mask_3tj_0L]
sm_events_3tj_truth = events_3tj_truth_0L[no_dupes_bool_0L][sm_mask_3tj_0L]

sm_truth_3tj_0L = {}
sm_reco_3tj_0L = {}

for field in sm_events_3tj_truth.fields:
    sm_truth_3tj_0L[f"{field}"] = sm_events_3tj_truth[field]
for field in sm_events_3tj_reco.fields:
    sm_reco_3tj_0L[f"{field}"] = sm_events_3tj_reco[field]

with  uproot.recreate(f"sm_3tj_0L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tj_0L
       file["Reco"] = sm_reco_3tj_0L

#### 1 Lepton Channel

In [7]:
matched_events_3tW_1L, no_dupes_bool_1L = truth_matching(truth_3tW_1L, truth_id_3tW_1L, reco_3tW_jets_1L, reco_3tW_leptons_1L)

matched_3tW_1L = ak.zip({'reco': matched_events_3tW_1L['reco'], 'truth': matched_events_3tW_1L['truth']})
events_3tW_truth_1L = events_3tW_truth_1L[no_dupes_bool_1L]
events_3tW_reco_1L = events_3tW_reco_1L[no_dupes_bool_1L]
events_3tW_t_neut_1L = events_3tW_t_neut_1L[no_dupes_bool_1L]
events_3tW_r_neut_1L = events_3tW_r_neut_1L[no_dupes_bool_1L]
lepton_indices_1L = ak.sort(ak.argsort(abs(events_3tW_truth_1L['W_decay_id']), axis = 1, ascending=False)[:,:1], axis=1)

10
starting dm
starting axis=0 concat


In [8]:
sm_mask_3tW_1L, dm_mask_3tW_1L, fm_mask_3tW_1L= top_identifier(matched_3tW_1L, '3tW', '1L', lepton_indices=lepton_indices_1L)

In [9]:
events_3tW_truth_0_1L = events_3tW_truth_1L[(lepton_indices_1L[:,0]==0)]
events_3tW_truth_2_1L = events_3tW_truth_1L[(lepton_indices_1L[:,0]==2)]
events_3tW_truth_4_1L = events_3tW_truth_1L[(lepton_indices_1L[:,0]==4)]

events_3tW_reco_0_1L = events_3tW_reco_1L[(lepton_indices_1L[:,0]==0)]
events_3tW_reco_2_1L = events_3tW_reco_1L[(lepton_indices_1L[:,0]==2)]
events_3tW_reco_4_1L = events_3tW_reco_1L[(lepton_indices_1L[:,0]==4)]

events_3tW_t_neut_0_1L = events_3tW_t_neut_1L[(lepton_indices_1L[:,0]==0)]
events_3tW_t_neut_2_1L = events_3tW_t_neut_1L[(lepton_indices_1L[:,0]==2)]
events_3tW_t_neut_4_1L = events_3tW_t_neut_1L[(lepton_indices_1L[:,0]==4)]

events_3tW_r_neut_0_1L = events_3tW_r_neut_1L[(lepton_indices_1L[:,0]==0)]
events_3tW_r_neut_2_1L = events_3tW_r_neut_1L[(lepton_indices_1L[:,0]==2)]
events_3tW_r_neut_4_1L = events_3tW_r_neut_1L[(lepton_indices_1L[:,0]==4)]

In [10]:
events_3tW_truth_1L = ak.concatenate((events_3tW_truth_0_1L, events_3tW_truth_2_1L, events_3tW_truth_4_1L))

In [12]:
events_3tW_reco_1L = ak.concatenate((events_3tW_reco_0_1L, events_3tW_reco_2_1L, events_3tW_reco_4_1L))

In [11]:
events_3tW_t_neut_1L = ak.concatenate((events_3tW_t_neut_0_1L, events_3tW_t_neut_2_1L, events_3tW_t_neut_4_1L))
events_3tW_r_neut_1L = ak.concatenate((events_3tW_r_neut_0_1L, events_3tW_r_neut_2_1L, events_3tW_r_neut_4_1L))

In [13]:
fm_events_3tW_reco = events_3tW_reco_1L[fm_mask_3tW_1L]
fm_events_3tW_truth = events_3tW_truth_1L[fm_mask_3tW_1L]
fm_events_3tW_t_neut = events_3tW_t_neut_1L[fm_mask_3tW_1L]
fm_events_3tW_r_neut = events_3tW_r_neut_1L[fm_mask_3tW_1L]

fm_truth_3tW_1L = {}
fm_reco_3tW_1L = {}
fm_t_neut_3tW_1L = {}
fm_r_neut_3tW_1L = {}

for field in fm_events_3tW_truth.fields:
    fm_truth_3tW_1L[f"{field}"] = fm_events_3tW_truth[field]
for field in fm_events_3tW_reco.fields:
    fm_reco_3tW_1L[f"{field}"] = fm_events_3tW_reco[field]
for field in fm_events_3tW_t_neut.fields:
    fm_t_neut_3tW_1L[f"{field}"] = fm_events_3tW_t_neut[field]
for field in fm_events_3tW_r_neut.fields:
    fm_r_neut_3tW_1L[f"{field}"] = fm_events_3tW_r_neut[field]

with  uproot.recreate(f"fm_3tW_1L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tW_1L
       file["Reco"] = fm_reco_3tW_1L
       file["Truth;Neutrino"] = fm_t_neut_3tW_1L
       file["Reco;Neutrino"] = fm_r_neut_3tW_1L

dm_events_3tW_reco = events_3tW_reco_1L[dm_mask_3tW_1L]
dm_events_3tW_truth = events_3tW_truth_1L[dm_mask_3tW_1L]
dm_events_3tW_t_neut = events_3tW_t_neut_1L[dm_mask_3tW_1L]
dm_events_3tW_r_neut = events_3tW_r_neut_1L[dm_mask_3tW_1L]

dm_truth_3tW_1L = {}
dm_reco_3tW_1L = {}
dm_t_neut_3tW_1L = {}
dm_r_neut_3tW_1L = {}

for field in dm_events_3tW_truth.fields:
    dm_truth_3tW_1L[f"{field}"] = dm_events_3tW_truth[field]
for field in dm_events_3tW_reco.fields:
    dm_reco_3tW_1L[f"{field}"] = dm_events_3tW_reco[field]
for field in dm_events_3tW_t_neut.fields:
    dm_t_neut_3tW_1L[f"{field}"] = dm_events_3tW_t_neut[field]
for field in dm_events_3tW_r_neut.fields:
    dm_r_neut_3tW_1L[f"{field}"] = dm_events_3tW_r_neut[field]

with  uproot.recreate(f"dm_3tW_1L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tW_1L
       file["Reco"] = dm_reco_3tW_1L
       file["Truth;Neutrino"] = dm_t_neut_3tW_1L
       file["Reco;Neutrino"] = dm_r_neut_3tW_1L

sm_events_3tW_reco = events_3tW_reco_1L[sm_mask_3tW_1L]
sm_events_3tW_truth = events_3tW_truth_1L[sm_mask_3tW_1L]
sm_events_3tW_t_neut = events_3tW_t_neut_1L[sm_mask_3tW_1L]
sm_events_3tW_r_neut = events_3tW_r_neut_1L[sm_mask_3tW_1L]

sm_truth_3tW_1L = {}
sm_reco_3tW_1L = {}
sm_t_neut_3tW_1L = {}
sm_r_neut_3tW_1L = {}

for field in sm_events_3tW_truth.fields:
    sm_truth_3tW_1L[f"{field}"] = sm_events_3tW_truth[field]
for field in sm_events_3tW_reco.fields:
    sm_reco_3tW_1L[f"{field}"] = sm_events_3tW_reco[field]
for field in sm_events_3tW_t_neut.fields:
    sm_t_neut_3tW_1L[f"{field}"] = sm_events_3tW_t_neut[field]
for field in sm_events_3tW_r_neut.fields:
    sm_r_neut_3tW_1L[f"{field}"] = sm_events_3tW_r_neut[field]

with  uproot.recreate(f"sm_3tW_1L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tW_1L
       file["Reco"] = sm_reco_3tW_1L
       file["Truth;Neutrino"] = sm_t_neut_3tW_1L
       file["Reco;Neutrino"] = sm_r_neut_3tW_1L

In [8]:
matched_events_3tj_1L, no_dupes_bool_1L = truth_matching(truth_3tj_1L, truth_id_3tj_1L, reco_3tj_jets_1L, reco_3tj_leptons_1L)

matched_3tj_1L = ak.zip({'reco': matched_events_3tj_1L['reco'], 'truth': matched_events_3tj_1L['truth']})
events_3tj_truth_1L = events_3tj_truth_1L[no_dupes_bool_1L]
events_3tj_reco_1L = events_3tj_reco_1L[no_dupes_bool_1L]
events_3tj_t_neut_1L = events_3tj_t_neut_1L[no_dupes_bool_1L]
events_3tj_r_neut_1L = events_3tj_r_neut_1L[no_dupes_bool_1L]
lepton_indices_1L = ak.sort(ak.argsort(abs(events_3tj_truth_1L['W_decay_id']), axis = 1, ascending=False)[:,:1], axis=1)

8


starting dm
starting axis=0 concat


In [9]:
sm_mask_3tj_1L, dm_mask_3tj_1L, fm_mask_3tj_1L= top_identifier(matched_3tj_1L, '3tj', '1L', lepton_indices=lepton_indices_1L)

In [10]:
events_3tj_truth_0_1L = events_3tj_truth_1L[(lepton_indices_1L[:,0]==0)]
events_3tj_truth_2_1L = events_3tj_truth_1L[(lepton_indices_1L[:,0]==2)]
events_3tj_truth_4_1L = events_3tj_truth_1L[(lepton_indices_1L[:,0]==4)]

events_3tj_reco_0_1L = events_3tj_reco_1L[(lepton_indices_1L[:,0]==0)]
events_3tj_reco_2_1L = events_3tj_reco_1L[(lepton_indices_1L[:,0]==2)]
events_3tj_reco_4_1L = events_3tj_reco_1L[(lepton_indices_1L[:,0]==4)]

events_3tj_t_neut_0_1L = events_3tj_t_neut_1L[(lepton_indices_1L[:,0]==0)]
events_3tj_t_neut_2_1L = events_3tj_t_neut_1L[(lepton_indices_1L[:,0]==2)]
events_3tj_t_neut_4_1L = events_3tj_t_neut_1L[(lepton_indices_1L[:,0]==4)]

events_3tj_r_neut_0_1L = events_3tj_r_neut_1L[(lepton_indices_1L[:,0]==0)]
events_3tj_r_neut_2_1L = events_3tj_r_neut_1L[(lepton_indices_1L[:,0]==2)]
events_3tj_r_neut_4_1L = events_3tj_r_neut_1L[(lepton_indices_1L[:,0]==4)]

In [11]:
events_3tj_truth_1L = ak.concatenate((events_3tj_truth_0_1L, events_3tj_truth_2_1L, events_3tj_truth_4_1L))

In [12]:
events_3tj_reco_1L = ak.concatenate((events_3tj_reco_0_1L, events_3tj_reco_2_1L, events_3tj_reco_4_1L))

In [13]:
events_3tj_t_neut_1L = ak.concatenate((events_3tj_t_neut_0_1L, events_3tj_t_neut_2_1L, events_3tj_t_neut_4_1L))
events_3tj_r_neut_1L = ak.concatenate((events_3tj_r_neut_0_1L, events_3tj_r_neut_2_1L, events_3tj_r_neut_4_1L))

In [14]:
fm_events_3tj_reco = events_3tj_reco_1L[fm_mask_3tj_1L]
fm_events_3tj_truth = events_3tj_truth_1L[fm_mask_3tj_1L]
fm_events_3tj_t_neut = events_3tj_t_neut_1L[fm_mask_3tj_1L]
fm_events_3tj_r_neut = events_3tj_r_neut_1L[fm_mask_3tj_1L]

fm_truth_3tj_1L = {}
fm_reco_3tj_1L = {}
fm_t_neut_3tj_1L = {}
fm_r_neut_3tj_1L = {}

for field in fm_events_3tj_truth.fields:
    fm_truth_3tj_1L[f"{field}"] = fm_events_3tj_truth[field]
for field in fm_events_3tj_reco.fields:
    fm_reco_3tj_1L[f"{field}"] = fm_events_3tj_reco[field]
for field in fm_events_3tj_t_neut.fields:
    fm_t_neut_3tj_1L[f"{field}"] = fm_events_3tj_t_neut[field]
for field in fm_events_3tj_r_neut.fields:
    fm_r_neut_3tj_1L[f"{field}"] = fm_events_3tj_r_neut[field]

with  uproot.recreate(f"fm_3tj_1L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tj_1L
       file["Reco"] = fm_reco_3tj_1L
       file["Truth;Neutrino"] = fm_t_neut_3tj_1L
       file["Reco;Neutrino"] = fm_r_neut_3tj_1L

dm_events_3tj_reco = events_3tj_reco_1L[dm_mask_3tj_1L]
dm_events_3tj_truth = events_3tj_truth_1L[dm_mask_3tj_1L]
dm_events_3tj_t_neut = events_3tj_t_neut_1L[dm_mask_3tj_1L]
dm_events_3tj_r_neut = events_3tj_r_neut_1L[dm_mask_3tj_1L]

dm_truth_3tj_1L = {}
dm_reco_3tj_1L = {}
dm_t_neut_3tj_1L = {}
dm_r_neut_3tj_1L = {}

for field in dm_events_3tj_truth.fields:
    dm_truth_3tj_1L[f"{field}"] = dm_events_3tj_truth[field]
for field in dm_events_3tj_reco.fields:
    dm_reco_3tj_1L[f"{field}"] = dm_events_3tj_reco[field]
for field in dm_events_3tj_t_neut.fields:
    dm_t_neut_3tj_1L[f"{field}"] = dm_events_3tj_t_neut[field]
for field in dm_events_3tj_r_neut.fields:
    dm_r_neut_3tj_1L[f"{field}"] = dm_events_3tj_r_neut[field]

with  uproot.recreate(f"dm_3tj_1L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tj_1L
       file["Reco"] = dm_reco_3tj_1L
       file["Truth;Neutrino"] = dm_t_neut_3tj_1L
       file["Reco;Neutrino"] = dm_r_neut_3tj_1L

sm_events_3tj_reco = events_3tj_reco_1L[sm_mask_3tj_1L]
sm_events_3tj_truth = events_3tj_truth_1L[sm_mask_3tj_1L]
sm_events_3tj_t_neut = events_3tj_t_neut_1L[sm_mask_3tj_1L]
sm_events_3tj_r_neut = events_3tj_r_neut_1L[sm_mask_3tj_1L]

sm_truth_3tj_1L = {}
sm_reco_3tj_1L = {}
sm_t_neut_3tj_1L = {}
sm_r_neut_3tj_1L = {}

for field in sm_events_3tj_truth.fields:
    sm_truth_3tj_1L[f"{field}"] = sm_events_3tj_truth[field]
for field in sm_events_3tj_reco.fields:
    sm_reco_3tj_1L[f"{field}"] = sm_events_3tj_reco[field]
for field in sm_events_3tj_t_neut.fields:
    sm_t_neut_3tj_1L[f"{field}"] = sm_events_3tj_t_neut[field]
for field in sm_events_3tj_r_neut.fields:
    sm_r_neut_3tj_1L[f"{field}"] = sm_events_3tj_r_neut[field]

with  uproot.recreate(f"sm_3tj_1L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tj_1L
       file["Reco"] = sm_reco_3tj_1L
       file["Truth;Neutrino"] = sm_t_neut_3tj_1L
       file["Reco;Neutrino"] = sm_r_neut_3tj_1L

#### 2 Lepton Channel

In [14]:
matched_events_3tW_2L, no_dupes_bool_2L = truth_matching(truth_3tW_2L, truth_id_3tW_2L, reco_3tW_jets_2L, reco_3tW_leptons_2L)

matched_3tW_2L = ak.zip({'reco': matched_events_3tW_2L['reco'], 'truth': matched_events_3tW_2L['truth']})
events_3tW_truth_2L = events_3tW_truth_2L[no_dupes_bool_2L]
events_3tW_reco_2L = events_3tW_reco_2L[no_dupes_bool_2L]
lepton_indices_2L = ak.sort(ak.argsort(abs(events_3tW_truth_2L['W_decay_id']), axis = 1, ascending=False)[:,:2], axis=1)

9
starting dm
starting axis=0 concat


In [15]:
sm_mask_3tW_2L, dm_mask_3tW_2L, fm_mask_3tW_2L= top_identifier(matched_3tW_2L, '3tW', '2L', lepton_indices=lepton_indices_2L)

In [17]:
events_3tW_truth_01_2L = events_3tW_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tW_truth_03_2L = events_3tW_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tW_truth_23_2L = events_3tW_truth_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

events_3tW_reco_01_2L = events_3tW_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tW_reco_03_2L = events_3tW_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tW_reco_23_2L = events_3tW_reco_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

In [18]:
events_3tW_truth_2L = ak.concatenate((events_3tW_truth_01_2L, events_3tW_truth_03_2L, events_3tW_truth_23_2L))

In [19]:
events_3tW_reco_2L = ak.concatenate((events_3tW_reco_01_2L, events_3tW_reco_03_2L, events_3tW_reco_23_2L))

In [20]:
fm_events_3tW_reco = events_3tW_reco_2L[fm_mask_3tW_2L]
fm_events_3tW_truth = events_3tW_truth_2L[fm_mask_3tW_2L]

fm_truth_3tW_2L = {}
fm_reco_3tW_2L = {}

for field in fm_events_3tW_truth.fields:
    fm_truth_3tW_2L[f"{field}"] = fm_events_3tW_truth[field]
for field in fm_events_3tW_reco.fields:
    fm_reco_3tW_2L[f"{field}"] = fm_events_3tW_reco[field]

with  uproot.recreate(f"fm_3tW_2L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tW_2L
       file["Reco"] = fm_reco_3tW_2L

dm_events_3tW_reco = events_3tW_reco_2L[dm_mask_3tW_2L]
dm_events_3tW_truth = events_3tW_truth_2L[dm_mask_3tW_2L]

dm_truth_3tW_2L = {}
dm_reco_3tW_2L = {}

for field in dm_events_3tW_truth.fields:
    dm_truth_3tW_2L[f"{field}"] = dm_events_3tW_truth[field]
for field in dm_events_3tW_reco.fields:
    dm_reco_3tW_2L[f"{field}"] = dm_events_3tW_reco[field]

with  uproot.recreate(f"dm_3tW_2L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tW_2L
       file["Reco"] = dm_reco_3tW_2L

sm_events_3tW_reco = events_3tW_reco_2L[sm_mask_3tW_2L]
sm_events_3tW_truth = events_3tW_truth_2L[sm_mask_3tW_2L]

sm_truth_3tW_2L = {}
sm_reco_3tW_2L = {}

for field in sm_events_3tW_truth.fields:
    sm_truth_3tW_2L[f"{field}"] = sm_events_3tW_truth[field]
for field in sm_events_3tW_reco.fields:
    sm_reco_3tW_2L[f"{field}"] = sm_events_3tW_reco[field]

with  uproot.recreate(f"sm_3tW_2L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tW_2L
       file["Reco"] = sm_reco_3tW_2L

In [18]:
events_3tj_truth_2L

<Array [{b_id: [5, ...], b_pt: ..., ...}, ...] type='294959 * {b_id: var * ...'>

In [20]:
no_dupes_bool_2L[no_dupes_bool_2L]

<Array [True, True, True, True, ..., True, True, True] type='294959 * bool'>

In [15]:
matched_events_3tj_2L, no_dupes_bool_2L = truth_matching(truth_3tj_2L, truth_id_3tj_2L, reco_3tj_jets_2L, reco_3tj_leptons_2L)

matched_3tj_2L = ak.zip({'reco': matched_events_3tj_2L['reco'], 'truth': matched_events_3tj_2L['truth']})
events_3tj_truth_2L = events_3tj_truth_2L[no_dupes_bool_2L]
events_3tj_reco_2L = events_3tj_reco_2L[no_dupes_bool_2L]
lepton_indices_2L = ak.sort(ak.argsort(abs(events_3tj_truth_2L['W_decay_id']), axis = 1, ascending=False)[:,:2], axis=1)

sm_mask_3tj_2L, dm_mask_3tj_2L, fm_mask_3tj_2L= top_identifier(matched_3tj_2L, '3tj', '2L', lepton_indices=lepton_indices_2L)

events_3tj_truth_01_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tj_truth_03_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tj_truth_23_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

events_3tj_reco_01_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tj_reco_03_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tj_reco_23_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

7
starting dm
starting axis=0 concat


In [16]:
sm_mask_3tj_2L, dm_mask_3tj_2L, fm_mask_3tj_2L= top_identifier(matched_3tj_2L, '3tj', '2L', lepton_indices=lepton_indices_2L)

events_3tj_truth_01_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tj_truth_03_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tj_truth_23_2L = events_3tj_truth_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

events_3tj_reco_01_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==1)]
events_3tj_reco_03_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==0)&(lepton_indices_2L[:,1]==3)]
events_3tj_reco_23_2L = events_3tj_reco_2L[(lepton_indices_2L[:,0]==2)&(lepton_indices_2L[:,1]==3)]

In [17]:
events_3tj_truth_2L = ak.concatenate((events_3tj_truth_01_2L, events_3tj_truth_03_2L, events_3tj_truth_23_2L))
events_3tj_reco_2L = ak.concatenate((events_3tj_reco_01_2L, events_3tj_reco_03_2L, events_3tj_reco_23_2L))

In [18]:
fm_events_3tj_reco = events_3tj_reco_2L[fm_mask_3tj_2L]
fm_events_3tj_truth = events_3tj_truth_2L[fm_mask_3tj_2L]

fm_truth_3tj_2L = {}
fm_reco_3tj_2L = {}

for field in fm_events_3tj_truth.fields:
    fm_truth_3tj_2L[f"{field}"] = fm_events_3tj_truth[field]
for field in fm_events_3tj_reco.fields:
    fm_reco_3tj_2L[f"{field}"] = fm_events_3tj_reco[field]

with  uproot.recreate(f"fm_3tj_2L_02Dec.root") as file:
       file["Truth"] = fm_truth_3tj_2L
       file["Reco"] = fm_reco_3tj_2L

dm_events_3tj_reco = events_3tj_reco_2L[dm_mask_3tj_2L]
dm_events_3tj_truth = events_3tj_truth_2L[dm_mask_3tj_2L]

dm_truth_3tj_2L = {}
dm_reco_3tj_2L = {}

for field in dm_events_3tj_truth.fields:
    dm_truth_3tj_2L[f"{field}"] = dm_events_3tj_truth[field]
for field in dm_events_3tj_reco.fields:
    dm_reco_3tj_2L[f"{field}"] = dm_events_3tj_reco[field]

with  uproot.recreate(f"dm_3tj_2L_02Dec.root") as file:
       file["Truth"] = dm_truth_3tj_2L
       file["Reco"] = dm_reco_3tj_2L

sm_events_3tj_reco = events_3tj_reco_2L[sm_mask_3tj_2L]
sm_events_3tj_truth = events_3tj_truth_2L[sm_mask_3tj_2L]

sm_truth_3tj_2L = {}
sm_reco_3tj_2L = {}

for field in sm_events_3tj_truth.fields:
    sm_truth_3tj_2L[f"{field}"] = sm_events_3tj_truth[field]
for field in sm_events_3tj_reco.fields:
    sm_reco_3tj_2L[f"{field}"] = sm_events_3tj_reco[field]

with  uproot.recreate(f"sm_3tj_2L_02Dec.root") as file:
       file["Truth"] = sm_truth_3tj_2L
       file["Reco"] = sm_reco_3tj_2L